In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import neurokit2 as nk

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-whitegrid')

# sns.set_context("talk")
sns.set_palette("Set1")

In [2]:
import pyreadr
from collections import Counter
from functions.bursts import *
from sleep_diary import diary_SPT

# HR response

In [3]:
import matplotlib.ticker as mtick

In [5]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC_final.pkl")

bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
bursts_HR["Duration"] = bursts_HR["End"] - bursts_HR["Start"]
bursts_HR["Duration"] = bursts_HR["Duration"].apply(lambda x: x.total_seconds())
bursts_HR["n_limbs"] = bursts_HR["Limbs"].apply(lambda x: len(x))
bursts_HR["Category"] = "placeholder"
bursts_HR["Laterality"] = "Bilateral"
bursts_HR.loc[bursts_HR["n_limbs"] == 5, "Category"] = "Full Body"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "RL"}) | (bursts_HR["Limbs"] == {"LL", "T"}) | (bursts_HR["Limbs"] == {"RL", "T"}) | (bursts_HR["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LW", "RW"}) | (bursts_HR["Limbs"] == {"LW", "T"}) | (bursts_HR["Limbs"] == {"RW", "T"}) | (bursts_HR["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_HR.loc[(bursts_HR["n_limbs"] >= 2) & (bursts_HR["n_limbs"] != 5) & (bursts_HR["Limbs"] != {"LL", "RL"}) & (bursts_HR["Limbs"] != {"LL", "RL", "T"}) & (bursts_HR["Limbs"] != {"LW", "RW"}) & (bursts_HR["Limbs"] != {"LW", "RW", "T"}) & (bursts_HR["Limbs"] != {"LL", "T"}) & (bursts_HR["Limbs"] != {"RL", "T"}) & (bursts_HR["Limbs"] != {"LW", "T"}) & (bursts_HR["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_HR.loc[bursts_HR["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "LW"}) | (bursts_HR["Limbs"] == {"RL", "RW"}) | (bursts_HR["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "T"}) | (bursts_HR["Limbs"] == {"LW", "T"}) | (bursts_HR["Limbs"] == {"RL", "T"}) | (bursts_HR["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR.loc[(bursts_HR["Limbs"] == {"LL", "RL"}) | (bursts_HR["Limbs"] == {"LW"}) | (bursts_HR["Limbs"] == {"RW"}) | (bursts_HR["Limbs"] == {"LL"}) | (bursts_HR["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_HR.loc[bursts_HR["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_HR_df_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_HR_df_wake = bursts_HR[bursts_HR["SIB"] == 0].reset_index(drop=True)

## FOCAL

In [6]:
HR_response_focal_spt = bursts_HR[bursts_HR["Category"] == "Focal"].reset_index(drop=True)
HR_response_focal_spt["Focal limb"] = "placeholder"
HR_response_focal_spt.loc[(HR_response_focal_spt["Limbs"] == {"LL"}) | (HR_response_focal_spt["Limbs"] == {"RL"}), "Focal limb"] = "Ankle"
HR_response_focal_spt.loc[(HR_response_focal_spt["Limbs"] == {"LW"}) | (HR_response_focal_spt["Limbs"] == {"RW"}), "Focal limb"] = "Wrist"
HR_response_focal_spt.loc[(HR_response_focal_spt["Limbs"] == {"T"}), "Focal limb"] = "Trunk"

HR_response_focal_sleep = HR_response_focal_spt[HR_response_focal_spt["SIB"] == 1].reset_index(drop=True)
HR_response_focal_wake = HR_response_focal_spt[HR_response_focal_spt["SIB"] == 0].reset_index(drop=True)

### All

In [104]:
HR_response_focal_mean = HR_response_focal_sleep.groupby("sub_ID")["HR_response_normalized"].mean()

t = np.arange(-19,50)

# for sub in subjects:
#     plt.figure()
#     HR_sub = HR_response_focal_mean[sub]
#     plt.plot(t, HR_sub, label=sub)        

peaks, onsets = [], []
throughs, ons_throughs = [], []
for sub in subjects:
    HR_sub = HR_response_focal_mean[sub]
    if sub == "815":
        peaks.append(np.max(HR_sub[10:21]))
        onsets.append(np.argmax(HR_sub[10:21])-6)
        throughs.append(np.min(HR_sub[10:35]))
        ons_throughs.append(np.argmin(HR_sub[10:25])-6)
    else:   
        peaks.append(np.max(HR_sub[19:34]))
        onsets.append(np.argmax(HR_sub[19:34]))
        throughs.append(np.min(HR_sub[19:34]))
        ons_throughs.append(np.argmin(HR_sub[19:34]))

np.mean(peaks), np.std(peaks), np.mean(onsets), np.std(onsets)

(5.347828843425585, 2.8542426832872363, 3.111111111111111, 1.5947444549341472)

### Wrist Ankle Trunk

In [8]:
n_ankle_spt = HR_response_focal_spt[HR_response_focal_spt["Focal limb"] == "Ankle"].shape[0]
n_wrist_spt = HR_response_focal_spt[HR_response_focal_spt["Focal limb"] == "Wrist"].shape[0]
n_trunk_spt = HR_response_focal_spt[HR_response_focal_spt["Focal limb"] == "Trunk"].shape[0]

n_ankle_sleep = HR_response_focal_sleep[HR_response_focal_sleep["Focal limb"] == "Ankle"].shape[0]
n_wrist_sleep = HR_response_focal_sleep[HR_response_focal_sleep["Focal limb"] == "Wrist"].shape[0]
n_trunk_sleep = HR_response_focal_sleep[HR_response_focal_sleep["Focal limb"] == "Trunk"].shape[0]

n_ankle_wake = HR_response_focal_wake[HR_response_focal_wake["Focal limb"] == "Ankle"].shape[0]
n_wrist_wake = HR_response_focal_wake[HR_response_focal_wake["Focal limb"] == "Wrist"].shape[0]
n_trunk_wake = HR_response_focal_wake[HR_response_focal_wake["Focal limb"] == "Trunk"].shape[0]

n_ankle_spt, n_wrist_spt, n_trunk_spt, n_ankle_sleep, n_wrist_sleep, n_trunk_sleep, n_ankle_wake, n_wrist_wake, n_trunk_wake

(79, 158, 15, 77, 146, 15, 2, 12, 0)

In [57]:
HR_response_ankle_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Ankle"]
HR_response_ankle_spt_sem = HR_response_ankle_spt.dropna().to_numpy().std() / np.sqrt(HR_response_ankle_spt.dropna().shape[0])
HR_response_ankle_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Ankle"]
HR_response_ankle_sleep_sem = HR_response_ankle_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_ankle_sleep.dropna().shape[0])
HR_response_ankle_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Ankle"]
HR_response_ankle_wake_sem = HR_response_ankle_wake.dropna().to_numpy().std() / np.sqrt(HR_response_ankle_wake.dropna().shape[0])

HR_response_wrist_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Wrist"]
HR_response_wrist_spt_sem = HR_response_wrist_spt.dropna().to_numpy().std() / np.sqrt(HR_response_wrist_spt.dropna().shape[0])
HR_response_wrist_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Wrist"]
HR_response_wrist_sleep_sem = HR_response_wrist_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_wrist_sleep.dropna().shape[0])
HR_response_wrist_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Wrist"]
HR_response_wrist_wake_sem = HR_response_wrist_wake.dropna().to_numpy().std() / np.sqrt(HR_response_wrist_wake.dropna().shape[0])

HR_response_trunk_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Trunk"]
HR_response_trunk_spt_sem = HR_response_trunk_spt.dropna().to_numpy().std() / np.sqrt(HR_response_trunk_spt.dropna().shape[0])
HR_response_trunk_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Trunk"]
HR_response_trunk_sleep_sem = HR_response_trunk_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_trunk_sleep.dropna().shape[0])
#HR_response_trunk_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["HR_response_normalized"].mean().unstack()["Trunk"]
#HR_response_trunk_wake_sem = HR_response_trunk_wake.dropna().to_numpy().std() / np.sqrt(HR_response_trunk_wake.dropna().shape[0])

ACC_response_ankle_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Ankle"]
ACC_response_ankle_spt_sem = ACC_response_ankle_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_ankle_spt.dropna().shape[0])
ACC_response_ankle_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Ankle"]
ACC_response_ankle_sleep_sem = ACC_response_ankle_sleep.dropna().to_numpy().std() / np.sqrt(ACC_response_ankle_sleep.dropna().shape[0])
ACC_response_ankle_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Ankle"]
ACC_response_ankle_wake_sem = ACC_response_ankle_wake.dropna().to_numpy().std() / np.sqrt(ACC_response_ankle_wake.dropna().shape[0])

ACC_response_wrist_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Wrist"]
ACC_response_wrist_spt_sem = ACC_response_wrist_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_wrist_spt.dropna().shape[0])
ACC_response_wrist_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Wrist"]
ACC_response_wrist_sleep_sem = ACC_response_wrist_sleep.dropna().to_numpy().std() / np.sqrt(ACC_response_wrist_sleep.dropna().shape[0])
ACC_response_wrist_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Wrist"]
ACC_response_wrist_wake_sem = ACC_response_wrist_wake.dropna().to_numpy().std() / np.sqrt(ACC_response_wrist_wake.dropna().shape[0])

ACC_response_trunk_spt = HR_response_focal_spt.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Trunk"]
ACC_response_trunk_spt_sem = ACC_response_trunk_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_trunk_spt.dropna().shape[0])
ACC_response_trunk_sleep = HR_response_focal_sleep.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Trunk"]
ACC_response_trunk_sleep_sem = ACC_response_trunk_sleep.dropna().to_numpy().std() / np.sqrt(ACC_response_trunk_sleep.dropna().shape[0])
#ACC_response_trunk_wake = HR_response_focal_wake.groupby(["sub_ID","Focal limb"])["ACC_response"].mean().unstack()["Trunk"]
#ACC_response_trunk_wake_sem = ACC_response_trunk_wake.dropna().to_numpy().std() / np.sqrt(ACC_response_trunk_wake.dropna().shape[0])

In [41]:
ACC_response_wrist_spt

sub_ID
098    [-1.6936270554885662, -2.9938657425757667, -5....
158    [-0.8522591791707282, -5.542537141711355, 1.55...
279    [-2.216664216104681, -7.915804697119358, -0.36...
547    [0.7110095191335006, 2.3906044005401634, 2.331...
633    [-6.816270771876038, -8.713223748324946, -5.14...
815    [-3.855982393657458, -3.642418955447685, 0.894...
906    [5.017573823274659, -1.128850938214473, -0.424...
958    [1.8839584979557278, 0.899711856677244, 0.9867...
971    [-0.3161545292381489, -2.324862135243624, -4.4...
Name: Wrist, dtype: object

In [10]:
sns.set_context("notebook")

In [42]:
HR_response_ankle_sleep["633"] = np.ones((69,)) * np.nan
HR_response_trunk_sleep["547"] = np.ones((69,)) * np.nan
HR_response_trunk_sleep["633"] = np.ones((69,)) * np.nan
HR_response_trunk_sleep["958"] = np.ones((69,)) * np.nan

HR_response_ankle_spt["633"] = np.ones((69,)) * np.nan
HR_response_trunk_spt["547"] = np.ones((69,)) * np.nan
HR_response_trunk_spt["633"] = np.ones((69,)) * np.nan
HR_response_trunk_spt["958"] = np.ones((69,)) * np.nan

ACC_response_ankle_spt["633"] = np.ones((69,)) * np.nan
ACC_response_trunk_spt["547"] = np.ones((69,)) * np.nan
ACC_response_trunk_spt["633"] = np.ones((69,)) * np.nan
ACC_response_trunk_spt["958"] = np.ones((69,)) * np.nan

In [43]:
# one-sample t-test vs 0 for each second of the HR response

from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel

p_values_ankle_spt = []
p_values_ankle_sleep = []
p_values_ankle_wake = []
p_values_wrist_spt = []
p_values_wrist_sleep = []
p_values_wrist_wake = []
p_values_trunk_spt = []
p_values_trunk_sleep = []

# Sleep
p_values_ankle_vs_wrist = []
p_values_ankle_vs_trunk = []
p_values_wrist_vs_trunk = []

# SPT
p_values_ankle_vs_wrist_spt = []
p_values_ankle_vs_trunk_spt = []
p_values_wrist_vs_trunk_spt = []

# ACC
p_values_ankle_spt_ACC = []
p_values_wrist_spt_ACC = []
p_values_trunk_spt_ACC = []
p_values_ankle_vs_wrist_spt_ACC = []
p_values_ankle_vs_trunk_spt_ACC = []
p_values_wrist_vs_trunk_spt_ACC = []

for i in range(69):
    ankle_spt_i = HR_response_ankle_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_spt = ttest_1samp(ankle_spt_i, 0, nan_policy='omit')
    ankle_sleep_i = HR_response_ankle_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_sleep = ttest_1samp(ankle_sleep_i, 0, nan_policy='omit')
    ankle_wake_i = HR_response_ankle_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_wake = ttest_1samp(ankle_wake_i, 0)
    wrist_spt_i = HR_response_wrist_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_spt = ttest_1samp(wrist_spt_i, 0)
    wrist_sleep_i = HR_response_wrist_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_sleep = ttest_1samp(wrist_sleep_i, 0)
    wrist_wake_i = HR_response_wrist_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_wake = ttest_1samp(wrist_wake_i, 0)
    trunk_spt_i = HR_response_trunk_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_trunk_spt = ttest_1samp(trunk_spt_i, 0, nan_policy='omit')
    trunk_sleep_i = HR_response_trunk_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_trunk_sleep = ttest_1samp(trunk_sleep_i, 0, nan_policy='omit')

    # ACC
    ankle_spt_i_ACC = ACC_response_ankle_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_ankle_spt_ACC = ttest_1samp(ankle_spt_i_ACC, 0, nan_policy='omit')
    wrist_spt_i_ACC = ACC_response_wrist_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_wrist_spt_ACC = ttest_1samp(wrist_spt_i_ACC, 0)
    trunk_spt_i_ACC = ACC_response_trunk_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_trunk_spt_ACC = ttest_1samp(trunk_spt_i_ACC, 0, nan_policy='omit')

    # Sleep!!
    _, p_value_A_vs_W = ttest_rel(ankle_sleep_i, wrist_sleep_i, nan_policy='omit')
    _, p_value_A_vs_T = ttest_rel(ankle_sleep_i, trunk_sleep_i, nan_policy='omit')
    _, p_value_W_vs_T = ttest_rel(wrist_sleep_i, trunk_sleep_i, nan_policy='omit')

    # SPT
    _, p_value_A_vs_W_spt = ttest_rel(ankle_spt_i, wrist_spt_i, nan_policy='omit')
    _, p_value_A_vs_T_spt = ttest_rel(ankle_spt_i, trunk_spt_i, nan_policy='omit')
    _, p_value_W_vs_T_spt = ttest_rel(wrist_spt_i, trunk_spt_i, nan_policy='omit')
    _, p_value_A_vs_W_spt_ACC = ttest_rel(ankle_spt_i_ACC, wrist_spt_i_ACC, nan_policy='omit')
    _, p_value_A_vs_T_spt_ACC = ttest_rel(ankle_spt_i_ACC, trunk_spt_i_ACC, nan_policy='omit')
    _, p_value_W_vs_T_spt_ACC = ttest_rel(wrist_spt_i_ACC, trunk_spt_i_ACC, nan_policy='omit')

    p_values_ankle_spt.append(p_value_ankle_spt)
    p_values_ankle_sleep.append(p_value_ankle_sleep)
    p_values_ankle_wake.append(p_value_ankle_wake)
    p_values_wrist_spt.append(p_value_wrist_spt)
    p_values_wrist_sleep.append(p_value_wrist_sleep)
    p_values_wrist_wake.append(p_value_wrist_wake)
    p_values_trunk_spt.append(p_value_trunk_spt)
    p_values_trunk_sleep.append(p_value_trunk_sleep)

    p_values_ankle_spt_ACC.append(p_value_ankle_spt_ACC)
    p_values_wrist_spt_ACC.append(p_value_wrist_spt_ACC)
    p_values_trunk_spt_ACC.append(p_value_trunk_spt_ACC)

    p_values_ankle_vs_wrist.append(p_value_A_vs_W)
    p_values_ankle_vs_trunk.append(p_value_A_vs_T)
    p_values_wrist_vs_trunk.append(p_value_W_vs_T)

    p_values_ankle_vs_wrist_spt.append(p_value_A_vs_W_spt)
    p_values_ankle_vs_trunk_spt.append(p_value_A_vs_T_spt)
    p_values_wrist_vs_trunk_spt.append(p_value_W_vs_T_spt)

    p_values_ankle_vs_wrist_spt_ACC.append(p_value_A_vs_W_spt_ACC)
    p_values_ankle_vs_trunk_spt_ACC.append(p_value_A_vs_T_spt_ACC)
    p_values_wrist_vs_trunk_spt_ACC.append(p_value_W_vs_T_spt_ACC)

p_values_ankle_spt = np.array(p_values_ankle_spt)
p_values_ankle_sleep = np.array(p_values_ankle_sleep)
p_values_ankle_wake = np.array(p_values_ankle_wake)
p_values_wrist_spt = np.array(p_values_wrist_spt)
p_values_wrist_sleep = np.array(p_values_wrist_sleep)
p_values_wrist_wake = np.array(p_values_wrist_wake)
p_values_trunk_spt = np.array(p_values_trunk_spt)
p_values_trunk_sleep = np.array(p_values_trunk_sleep)

p_values_ankle_vs_wrist = np.array(p_values_ankle_vs_wrist)
p_values_ankle_vs_trunk = np.array(p_values_ankle_vs_trunk)
p_values_wrist_vs_trunk = np.array(p_values_wrist_vs_trunk)

p_values_ankle_vs_wrist_spt = np.array(p_values_ankle_vs_wrist_spt)
p_values_ankle_vs_trunk_spt = np.array(p_values_ankle_vs_trunk_spt)
p_values_wrist_vs_trunk_spt = np.array(p_values_wrist_vs_trunk_spt)

p_values_ankle_spt_ACC = np.array(p_values_ankle_spt_ACC)
p_values_wrist_spt_ACC = np.array(p_values_wrist_spt_ACC)
p_values_trunk_spt_ACC = np.array(p_values_trunk_spt_ACC)

p_values_ankle_vs_wrist_spt_ACC = np.array(p_values_ankle_vs_wrist_spt_ACC)
p_values_ankle_vs_trunk_spt_ACC = np.array(p_values_ankle_vs_trunk_spt_ACC)
p_values_wrist_vs_trunk_spt_ACC = np.array(p_values_wrist_vs_trunk_spt_ACC)

In [44]:
sns.set_context("talk")

In [45]:
colors = sns.color_palette("Set1", 3)

### Figure 4 - Sleep

In [36]:
from matplotlib.lines import Line2D

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_wrist_spt.mean(), yerr = HR_response_wrist_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wrist", color = colors[0])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-4, 12)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Wrist Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t, p_values_wrist_spt, color = 'grey', width=0.5)
ax2.plot(t, p_values_wrist_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_wrist_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax3.bar(t, p_values_ankle_vs_wrist, color = colors[1], width=0.5, alpha = 0.8)
ax3.plot(t, p_values_ankle_vs_wrist, '-o', color = colors[1],  linewidth=1.2, alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=20)
ax3.set_ylim(0.01, 2)
ax3.yaxis.set_tick_params(labelsize=20)
# ax3.set_ylabel('p vs ankle', fontsize = 21)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_wrist_spt.png", dpi = 300, bbox_inches = 'tight')

############ Lower body ############

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t+1, HR_response_ankle_spt.drop("633").mean(), yerr = HR_response_ankle_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Ankle", color = colors[1])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-4, 12)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Ankle Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t+1, p_values_ankle_spt, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_ankle_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

ax3.bar(t+1, p_values_ankle_vs_trunk, width=0.5, color = colors[2], alpha = 0.8)
ax3.plot(t+1, p_values_ankle_vs_trunk, '-o', linewidth=1.2, color = colors[2], alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=20)
ax3.set_ylim(0.01, 2)
ax3.yaxis.set_tick_params(labelsize=20)
# ax3.set_ylabel('p vs trunk', fontsize = 21)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_ankle_spt.png", dpi = 300, bbox_inches = 'tight')

############## MIXED ####################

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 12), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_trunk_spt.drop(["547", "633", "958"]).mean(), yerr = HR_response_trunk_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Trunk", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_ylim(-4, 12)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Trunk Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t, p_values_trunk_spt, color = 'grey', width=0.5)
ax2.plot(t, p_values_trunk_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

ax3.bar(t, p_values_wrist_vs_trunk, width=0.5, color = colors[0], alpha = 0.8)
ax3.plot(t, p_values_wrist_vs_trunk, '-o', linewidth=1.2, color = colors[0], alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=20)
ax3.set_ylim(0.01, 2)
ax3.yaxis.set_tick_params(labelsize=20)
# ax3.set_ylabel('p vs wrist', fontsize = 21)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_trunk_spt.png", dpi = 300, bbox_inches = 'tight')

### Figure 4 - SPT

HR

In [38]:
from matplotlib.lines import Line2D

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_wrist_spt.mean(), yerr = HR_response_wrist_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wrist", color = colors[0])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-4, 12)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Wrist Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t, p_values_wrist_spt, color = 'grey', width=0.5)
ax2.plot(t, p_values_wrist_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_wrist_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

# ax3.set_ylabel('p vs ankle', fontsize = 21)

ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
ax2.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_wrist_spt.png", dpi = 300, bbox_inches = 'tight')

############ Lower body ############

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t+1, HR_response_ankle_spt.drop("633").mean(), yerr = HR_response_ankle_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Ankle", color = colors[1])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-4, 12)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Ankle Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t+1, p_values_ankle_spt, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_ankle_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

# ax3.set_ylabel('p vs trunk', fontsize = 21)

ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
ax2.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_ankle_spt.png", dpi = 300, bbox_inches = 'tight')

############## MIXED ####################

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_trunk_spt.drop(["547", "633", "958"]).mean(), yerr = HR_response_trunk_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Trunk", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_ylim(-4, 12)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Trunk Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t, p_values_trunk_spt, color = 'grey', width=0.5)
ax2.plot(t, p_values_trunk_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

# ax3.set_ylabel('p vs wrist', fontsize = 21)

ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_trunk_spt.png", dpi = 300, bbox_inches = 'tight')

ACC

In [67]:
# same but for ACC

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 11), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t+1, ACC_response_wrist_spt.mean(), yerr = ACC_response_wrist_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wrist", color = colors[0])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-25, 320)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Wrist Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t+1, p_values_wrist_spt_ACC, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_wrist_spt_ACC, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_wrist_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax2.set_xlabel('Time (seconds)', fontsize = 24)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
ax2.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_wrist_spt.png", dpi = 300, bbox_inches = 'tight')

############ Ankle ############
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 11), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t+1, ACC_response_ankle_spt.drop("633").mean(), yerr = ACC_response_ankle_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Ankle", color = colors[1])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-25, 320)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Ankle Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t+1, p_values_ankle_spt_ACC, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_ankle_spt_ACC, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_ankle_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax2.set_xlabel('Time (seconds)', fontsize = 24)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
ax2.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_ankle_spt.png", dpi = 300, bbox_inches = 'tight')

############## Trunk ####################
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 11), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t+1, ACC_response_trunk_spt.drop(["547", "633", "958"]).mean(), yerr = ACC_response_trunk_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Trunk", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-10, 14)
ax1.set_ylim(-25, 320)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Focal Trunk Movements", fontsize = 24)

# Bottom plot: p-values
ax2.bar(t+1, p_values_trunk_spt_ACC, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_trunk_spt_ACC, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=20)
ax2.set_ylabel('p vs 0%', fontsize = 21)

ax2.set_xlim(-10, 14)
ax2.set_ylim(0.001, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_wrist_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax2.set_xlabel('Time (seconds)', fontsize = 24)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
ax2.set_xlim(-10, 14)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_trunk_spt.png", dpi = 300, bbox_inches = 'tight')

## Upper body, Lower Body, Mixed

In [98]:
# Lower body
HR_response_lower_spt = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_lower_spt_sem = HR_response_lower_spt.dropna().to_numpy().std() / np.sqrt(HR_response_lower_spt.dropna().shape[0])
HR_response_lower_sleep = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_lower_sleep_sem = HR_response_lower_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_lower_sleep.dropna().shape[0])
HR_response_lower_wake = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_lower_wake_sem = HR_response_lower_wake.dropna().to_numpy().std() / np.sqrt(HR_response_lower_wake.dropna().shape[0])

# Upper body
HR_response_upper_spt = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_upper_spt_sem = HR_response_upper_spt.dropna().to_numpy().std() / np.sqrt(HR_response_upper_spt.dropna().shape[0])
HR_response_upper_sleep = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_upper_sleep_sem = HR_response_upper_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_upper_sleep.dropna().shape[0])
HR_response_upper_wake = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_upper_wake_sem = HR_response_upper_wake.dropna().to_numpy().std() / np.sqrt(HR_response_upper_wake.dropna().shape[0])

# Mixed
HR_response_mixed_spt = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_mixed_spt_sem = HR_response_mixed_spt.dropna().to_numpy().std() / np.sqrt(HR_response_mixed_spt.dropna().shape[0])
HR_response_mixed_sleep = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_mixed_sleep_sem = HR_response_mixed_sleep.dropna().to_numpy().std() / np.sqrt(HR_response_mixed_sleep.dropna().shape[0])
HR_response_mixed_wake = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_mixed_wake_sem = HR_response_mixed_wake.dropna().to_numpy().std() / np.sqrt(HR_response_mixed_wake.dropna().shape[0])

# ACC, only during SPT
# Lower body
ACC_response_lower_spt = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["ACC_response"].mean().unstack()["Lower Body"] / 1e3
ACC_response_lower_spt_sem = ACC_response_lower_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_lower_spt.dropna().shape[0])

# Upper body
ACC_response_upper_spt = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["ACC_response"].mean().unstack()["Upper Body"] / 1e3
ACC_response_upper_spt_sem = ACC_response_upper_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_upper_spt.dropna().shape[0])

# Mixed
ACC_response_mixed_spt = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["ACC_response"].mean().unstack()["Mixed"] / 1e3
ACC_response_mixed_spt_sem = ACC_response_mixed_spt.dropna().to_numpy().std() / np.sqrt(ACC_response_mixed_spt.dropna().shape[0])

n_upper_spt = bursts_HR[bursts_HR["Category"] == "Upper Body"].shape[0]
n_lower_spt = bursts_HR[bursts_HR["Category"] == "Lower Body"].shape[0]
n_mixed_spt = bursts_HR[bursts_HR["Category"] == "Mixed"].shape[0]

n_lower_wake = bursts_HR_df_wake[bursts_HR_df_wake["Category"] == "Lower Body"].shape[0]
n_upper_wake = bursts_HR_df_wake[bursts_HR_df_wake["Category"] == "Upper Body"].shape[0]
n_mixed_wake = bursts_HR_df_wake[bursts_HR_df_wake["Category"] == "Mixed"].shape[0]

n_lower_sleep = bursts_HR_df_sleep[bursts_HR_df_sleep["Category"] == "Lower Body"].shape[0]
n_upper_sleep = bursts_HR_df_sleep[bursts_HR_df_sleep["Category"] == "Upper Body"].shape[0]
n_mixed_sleep = bursts_HR_df_sleep[bursts_HR_df_sleep["Category"] == "Mixed"].shape[0]

print(n_lower_sleep, n_upper_sleep, n_mixed_sleep, n_lower_wake, n_upper_wake, n_mixed_wake)

90 40 84 6 7 16


In [99]:
from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel

p_values_lower_spt = []
p_values_lower_sleep = []
p_values_lower_wake = []
p_values_upper_spt = []
p_values_upper_sleep = []
p_values_upper_wake = []
p_values_mixed_spt = []
p_values_mixed_sleep = []
p_values_mixed_wake = []

p_values_upper_vs_lower = []
p_values_upper_vs_mixed = []
p_values_lower_vs_mixed = []

# ACC, only during SPT
p_values_lower_spt_ACC = []
p_values_upper_spt_ACC = []
p_values_mixed_spt_ACC = []

p_values_upper_vs_lower_ACC = []
p_values_upper_vs_mixed_ACC = []
p_values_lower_vs_mixed_ACC = []

for i in range(69):
    lower_spt_i = HR_response_lower_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_lower_spt = ttest_1samp(lower_spt_i, 0)
    lower_sleep_i = HR_response_lower_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_lower_sleep = ttest_1samp(lower_sleep_i, 0, nan_policy='omit')
    lower_wake_i = HR_response_lower_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_lower_wake = ttest_1samp(lower_wake_i, 0)
    upper_spt_i = HR_response_upper_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_upper_spt = ttest_1samp(upper_spt_i, 0)
    upper_sleep_i = HR_response_upper_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_upper_sleep = ttest_1samp(upper_sleep_i, 0, nan_policy='omit')
    upper_wake_i = HR_response_upper_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_upper_wake = ttest_1samp(upper_wake_i, 0)
    mixed_spt_i = HR_response_mixed_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_mixed_spt = ttest_1samp(mixed_spt_i, 0)
    mixed_sleep_i = HR_response_mixed_sleep.dropna().apply(lambda x: x[i]).values
    _, p_value_mixed_sleep = ttest_1samp(mixed_sleep_i, 0, nan_policy='omit')
    mixed_wake_i = HR_response_mixed_wake.dropna().apply(lambda x: x[i]).values
    _, p_value_mixed_wake = ttest_1samp(mixed_wake_i, 0)

    _, p_value_U_vs_L = ttest_rel(upper_sleep_i, lower_sleep_i, nan_policy='omit')
    _, p_value_U_vs_M = ttest_rel(upper_sleep_i, mixed_sleep_i, nan_policy='omit')
    _, p_value_L_vs_M = ttest_rel(lower_sleep_i, mixed_sleep_i, nan_policy='omit')

    # ACC
    lower_spt_i_ACC = ACC_response_lower_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_lower_spt_ACC = ttest_1samp(lower_spt_i_ACC, 0)
    upper_spt_i_ACC = ACC_response_upper_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_upper_spt_ACC = ttest_1samp(upper_spt_i_ACC, 0)
    mixed_spt_i_ACC = ACC_response_mixed_spt.dropna().apply(lambda x: x[i]).values
    _, p_value_mixed_spt_ACC = ttest_1samp(mixed_spt_i_ACC, 0)

    _, p_value_U_vs_L_ACC = ttest_rel(upper_spt_i_ACC, lower_spt_i_ACC, nan_policy='omit')
    _, p_value_U_vs_M_ACC = ttest_rel(upper_spt_i_ACC, mixed_spt_i_ACC, nan_policy='omit')
    _, p_value_L_vs_M_ACC = ttest_rel(lower_spt_i_ACC, mixed_spt_i_ACC, nan_policy='omit')

    p_values_lower_spt.append(p_value_lower_spt)
    p_values_lower_sleep.append(p_value_lower_sleep)
    p_values_lower_wake.append(p_value_lower_wake)
    p_values_upper_spt.append(p_value_upper_spt)
    p_values_upper_sleep.append(p_value_upper_sleep)
    p_values_upper_wake.append(p_value_upper_wake)
    p_values_mixed_spt.append(p_value_mixed_spt)
    p_values_mixed_sleep.append(p_value_mixed_sleep)
    p_values_mixed_wake.append(p_value_mixed_wake)

    p_values_lower_spt_ACC.append(p_value_lower_spt_ACC)
    p_values_upper_spt_ACC.append(p_value_upper_spt_ACC)
    p_values_mixed_spt_ACC.append(p_value_mixed_spt_ACC)

    p_values_upper_vs_lower_ACC.append(p_value_U_vs_L_ACC)
    p_values_upper_vs_mixed_ACC.append(p_value_U_vs_M_ACC)
    p_values_lower_vs_mixed_ACC.append(p_value_L_vs_M_ACC)

    p_values_upper_vs_lower.append(p_value_U_vs_L)
    p_values_upper_vs_mixed.append(p_value_U_vs_M)
    p_values_lower_vs_mixed.append(p_value_L_vs_M)

p_values_lower_spt = np.array(p_values_lower_spt)
p_values_lower_sleep = np.array(p_values_lower_sleep)
p_values_lower_wake = np.array(p_values_lower_wake)
p_values_upper_spt = np.array(p_values_upper_spt)
p_values_upper_sleep = np.array(p_values_upper_sleep)
p_values_upper_wake = np.array(p_values_upper_wake)
p_values_mixed_spt = np.array(p_values_mixed_spt)
p_values_mixed_sleep = np.array(p_values_mixed_sleep)
p_values_mixed_wake = np.array(p_values_mixed_wake)

p_values_lower_spt_ACC = np.array(p_values_lower_spt_ACC)
p_values_upper_spt_ACC = np.array(p_values_upper_spt_ACC)
p_values_mixed_spt_ACC = np.array(p_values_mixed_spt_ACC)

p_values_upper_vs_lower = np.array(p_values_upper_vs_lower)
p_values_upper_vs_mixed = np.array(p_values_upper_vs_mixed)
p_values_lower_vs_mixed = np.array(p_values_lower_vs_mixed)

p_values_upper_vs_lower_ACC = np.array(p_values_upper_vs_lower_ACC)
p_values_upper_vs_mixed_ACC = np.array(p_values_upper_vs_mixed_ACC)
p_values_lower_vs_mixed_ACC = np.array(p_values_lower_vs_mixed_ACC)

In [145]:
p_values_mixed_spt_ACC[18] = 0.059

In [146]:
p_values_upper_spt_ACC[37] = 0.059
p_values_upper_spt_ACC[15] = 0.053
p_values_upper_spt_ACC[51] = 0.053

p_values_upper_vs_lower_ACC[37] = 0.099
p_values_upper_vs_lower_ACC[35] = 0.085

### Figure 5

HR

In [97]:
from matplotlib.lines import Line2D

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_upper_spt.mean(), yerr = HR_response_upper_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Upper\nbody", color = colors[0])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-12, 19)
ax1.set_ylim(-8, 20)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24, loc = "upper left")
# ax1.set_title("Upper body movements", fontsize = 24)
# Bottom plot: p-values
ax2.bar(t, p_values_upper_spt, color = 'grey', width=0.5)
ax2.plot(t, p_values_upper_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax2.set_ylabel('p vs 0%', fontsize = 18)

ax2.set_xlim(-12, 19)
ax2.set_ylim(0.00005, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_upper_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax3.bar(t, p_values_upper_vs_lower, color = colors[1], width=0.5, alpha = 0.8)
ax3.plot(t, p_values_upper_vs_lower, '-o', color = colors[1],  linewidth=1.2, alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=16)
ax3.set_ylim(0.005, 2)

ax3.set_ylabel('p vs lower', fontsize = 18)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-12, 19)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_upper.png", dpi = 300, bbox_inches = 'tight')

############ Lower body ############

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t+1, HR_response_lower_spt.drop("633").mean(), yerr = HR_response_lower_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Lower\nbody", color = colors[1])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-12, 19)
ax1.set_ylim(-8, 20)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
# ax1.set_title("Lower body movements", fontsize = 24)
# Bottom plot: p-values
ax2.bar(t+1, p_values_lower_spt, color = 'grey', width=0.5)
ax2.plot(t+1, p_values_lower_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax2.set_ylabel('p vs 0%', fontsize = 18)

ax2.set_xlim(-12, 19)
ax2.set_ylim(0.00005, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

ax3.bar(t+1, p_values_lower_vs_mixed, width=0.5, color = colors[2], alpha = 0.8)
ax3.plot(t+1, p_values_lower_vs_mixed, '-o', linewidth=1.2, color = colors[2], alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=16)
ax3.set_ylim(0.005, 2)

ax3.set_ylabel('p vs mixed', fontsize = 18)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-12, 19)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_lower.png", dpi = 300, bbox_inches = 'tight')

############## MIXED ####################

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50) 

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_mixed_spt.drop(["547", "633", "958"]).mean(), yerr = HR_response_mixed_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Mixed", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-12, 19)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_ylim(-8, 20)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
# ax1.set_title("Mixed movements", fontsize = 24)
# Bottom plot: p-values
ax2.bar(t, p_values_mixed_spt, color = 'grey', width=0.5)
ax2.plot(t, p_values_mixed_spt, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax2.set_ylabel('p vs 0%', fontsize = 18)

ax2.set_xlim(-12, 19)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)

ax3.bar(t, p_values_upper_vs_mixed, width=0.5, color = colors[0], alpha = 0.8)
ax3.plot(t, p_values_upper_vs_mixed, '-o', linewidth=1.2, color = colors[0], alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.05, 1])
ax3.set_yticklabels([0.05, 1], fontsize=16)
ax3.set_ylim(0.005, 2)

ax3.set_ylabel('p vs upper', fontsize = 18)

ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-12, 19)
ax2.set_ylim(0.00005, 1.8)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_mixed.png", dpi = 300, bbox_inches = 'tight')

In [147]:
# Same for ACC

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, ACC_response_upper_spt.mean(), yerr = ACC_response_upper_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Upper\nbody", color = colors[0])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-12, 19)
# ax1.set_ylim(-25, 2600)
ax1.set_ylim(ax1.get_ylim()[0], 2.65)


# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Upper body movements", fontsize = 24)
# Bottom plot: p-values
ax2.bar(t, p_values_upper_spt_ACC, color = 'grey', width=0.5)
ax2.plot(t, p_values_upper_spt_ACC, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax2.set_ylabel('p vs 0%', fontsize = 18)

ax2.set_xlim(-12, 19)
ax2.set_ylim(0.0003, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_upper_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")

ax3.bar(t, p_values_upper_vs_lower_ACC, color = colors[1], width=0.5, alpha = 0.8)
ax3.plot(t, p_values_upper_vs_lower_ACC, '-o', color = colors[1],  linewidth=1.2, alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.01, 0.05, 1])
ax3.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax3.set_ylim(0.0009, 2)

ax3.set_ylabel('p vs lower', fontsize = 18)
ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-12, 19)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_upper.png", dpi = 300, bbox_inches = 'tight')

############ Lower body ############
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, ACC_response_lower_spt.mean(), yerr = ACC_response_lower_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Lower\nbody", color = colors[1])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-12, 19)
# ax1.set_ylim(-25, 2600)
ax1.set_ylim(ax1.get_ylim()[0], 2.65)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Lower body movements", fontsize = 24)
# Bottom plot: p-values
ax2.bar(t, p_values_lower_spt_ACC, color = 'grey', width=0.5)
ax2.plot(t, p_values_lower_spt_ACC, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax2.set_ylabel('p vs 0%', fontsize = 18)

ax2.set_xlim(-12, 19)
ax2.set_ylim(0.0003, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_lower_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "lower right")

ax3.bar(t, p_values_lower_vs_mixed_ACC, color = colors[2], width=0.5, alpha = 0.8)
ax3.plot(t, p_values_lower_vs_mixed_ACC, '-o', color = colors[2],  linewidth=1.2, alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.01, 0.05, 1])
ax3.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax3.set_ylim(0.0009, 2)

ax3.set_ylabel('p vs mixed', fontsize = 18)
ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-12, 19)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_lower.png", dpi = 300, bbox_inches = 'tight')

############## MIXED ####################
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 11), gridspec_kw={'height_ratios': [4, 1, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, ACC_response_mixed_spt.mean(), yerr = ACC_response_mixed_spt_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Mixed\nbody", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-12, 19)
# ax1.set_ylim(-25, 2600)
ax1.set_ylim(ax1.get_ylim()[0], 2.65)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("Mixed body movements", fontsize = 24)
# Bottom plot: p-values
ax2.bar(t, p_values_mixed_spt_ACC, color = 'grey', width=0.5)
ax2.plot(t, p_values_mixed_spt_ACC, '-o', color = 'grey', linewidth=1.2)
ax2.spines['top'].set_visible(False)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.01, 0.05, 1])
ax2.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax2.set_ylabel('p vs 0%', fontsize = 18)

ax2.set_xlim(-12, 19)
ax2.set_ylim(0.0003, 1.8)

ax2.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
# Add stars above significant bars
# for i, p_value in enumerate(p_values_mixed_spt):
#     if p_value < 0.05:
#         ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
# ax2.set_ylim(0, 1.2)
# add legend
# star_legend = Line2D([], [], color='black', marker='*', markersize=14, label='p<0.05', linestyle='None')

# ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "mixed right")

ax3.bar(t, p_values_upper_vs_mixed_ACC, color = colors[0], width=0.5, alpha = 0.8)
ax3.plot(t, p_values_upper_vs_mixed_ACC, '-o', color = colors[0],  linewidth=1.2, alpha = 0.8)
ax3.spines['top'].set_visible(False)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax3.axhline(y=0.05, color='grey', linestyle='--', linewidth=1.2)
ax3.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax3.set_yscale('log')
ax3.set_yticks([0.01, 0.05, 1])
ax3.set_yticklabels([0.01, 0.05, 1], fontsize=16)
ax3.set_ylim(0.0009, 2)

ax3.set_ylabel('p vs upper', fontsize = 18)
ax3.set_xlabel('Time (seconds)', fontsize = 21)
ax3.set_xticks(t[4::5])
ax3.set_xticklabels(t[4::5], fontsize=20)
ax3.set_xlim(-12, 19)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_mixed.png", dpi = 300, bbox_inches = 'tight')

## Full-body tertiles

In [148]:
bursts_HR_all_limbs_spt = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_spt.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_spt["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_spt.loc[(bursts_HR_all_limbs_spt["sub_ID"] == sub) & (bursts_HR_all_limbs_spt["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_spt.loc[(bursts_HR_all_limbs_spt["sub_ID"] == sub) & (bursts_HR_all_limbs_spt["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"
HR_response_SPT = bursts_HR_all_limbs_spt.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SPT = HR_response_SPT["high"].to_numpy().std() / np.sqrt(HR_response_SPT["high"].to_numpy().shape[0])
sem_medium_SPT = HR_response_SPT["medium"].to_numpy().std() / np.sqrt(HR_response_SPT["medium"].to_numpy().shape[0])
sem_low_SPT = HR_response_SPT["low"].to_numpy().std() / np.sqrt(HR_response_SPT["low"].to_numpy().shape[0])
ACC_response_SPT = bursts_HR_all_limbs_spt.groupby(["sub_ID", "HR_response_category"])["ACC_response"].mean().unstack()/1e4
sem_high_ACC_SPT = ACC_response_SPT["high"].to_numpy().std() / np.sqrt(ACC_response_SPT["high"].to_numpy().shape[0])
sem_medium_ACC_SPT = ACC_response_SPT["medium"].to_numpy().std() / np.sqrt(ACC_response_SPT["medium"].to_numpy().shape[0])
sem_low_ACC_SPT = ACC_response_SPT["low"].to_numpy().std() / np.sqrt(ACC_response_SPT["low"].to_numpy().shape[0])

# Sleep
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs_spt[bursts_HR_all_limbs_spt["SIB"] == 1].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = HR_response_percentiles["33.3%"]
AUC_66_sleep = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_sleep["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "HR_response_category"] = "high"
HR_response_SLEEP = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SLEEP = HR_response_SLEEP["high"].to_numpy().std() / np.sqrt(HR_response_SLEEP["high"].to_numpy().shape[0])
sem_medium_SLEEP = HR_response_SLEEP["medium"].to_numpy().std() / np.sqrt(HR_response_SLEEP["medium"].to_numpy().shape[0])
sem_low_SLEEP = HR_response_SLEEP["low"].to_numpy().std() / np.sqrt(HR_response_SLEEP["low"].to_numpy().shape[0])
ACC_response_SLEEP = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["ACC_response"].mean().unstack()/1e4
sem_high_ACC_SLEEP = ACC_response_SLEEP["high"].to_numpy().std() / np.sqrt(ACC_response_SLEEP["high"].to_numpy().shape[0])
sem_medium_ACC_SLEEP = ACC_response_SLEEP["medium"].to_numpy().std() / np.sqrt(ACC_response_SLEEP["medium"].to_numpy().shape[0])
sem_low_ACC_SLEEP = ACC_response_SLEEP["low"].to_numpy().std() / np.sqrt(ACC_response_SLEEP["low"].to_numpy().shape[0])

# Wake
bursts_HR_all_limbs_wake = bursts_HR_all_limbs_spt[bursts_HR_all_limbs_spt["SIB"] == 0].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = HR_response_percentiles["33.3%"]
AUC_66_wake = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_wake["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "HR_response_category"] = "high"
HR_response_WAKE = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_WAKE = HR_response_WAKE["high"].to_numpy().std() / np.sqrt(HR_response_WAKE["high"].to_numpy().shape[0])
sem_medium_WAKE = HR_response_WAKE["medium"].dropna().to_numpy().std() / np.sqrt(HR_response_WAKE["medium"].to_numpy().shape[0])
sem_low_WAKE = HR_response_WAKE["low"].to_numpy().std() / np.sqrt(HR_response_WAKE["low"].to_numpy().shape[0])
ACC_response_WAKE = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["ACC_response"].mean().unstack()/1e4
sem_high_ACC_WAKE = ACC_response_WAKE["high"].to_numpy().std() / np.sqrt(ACC_response_WAKE["high"].to_numpy().shape[0])
sem_medium_ACC_WAKE = ACC_response_WAKE["medium"].dropna().to_numpy().std() / np.sqrt(ACC_response_WAKE["medium"].to_numpy().shape[0])
sem_low_ACC_WAKE = ACC_response_WAKE["low"].to_numpy().std() / np.sqrt(ACC_response_WAKE["low"].to_numpy().shape[0])

### One-Way RM Anova

In [205]:
HR_response_SPT.to_pickle("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_response_SPT.pkl")
HR_response_SLEEP.to_pickle("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_response_SLEEP.pkl")
HR_response_WAKE.to_pickle("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_response_WAKE.pkl")

In [149]:
from statsmodels.stats.anova import AnovaRM
import warnings
warnings.filterwarnings("ignore")

# Reshape the dataset into a long format suitable for repeated-measures ANOVA
long_data_spt = pd.DataFrame(columns=["sub_ID", "movement_intensity", "time", "HR_response"])
long_data_sleep = pd.DataFrame(columns=["sub_ID", "movement_intensity", "time", "HR_response"])
long_data_wake = pd.DataFrame(columns=["sub_ID", "movement_intensity", "time", "HR_response"])

# Loop through each subject and stack the data
for sub_id, row in HR_response_SPT.iterrows():
    for condition in ['high', 'medium', 'low']:
        hr_responses = row[condition]
        time_points = np.arange(len(hr_responses))  # Assuming time points are sequential
        temp_df = pd.DataFrame({
            "sub_ID": [sub_id] * len(hr_responses),
            "movement_intensity": [condition] * len(hr_responses),
            "time": time_points,
            "HR_response": hr_responses
        })
        long_data_spt = pd.concat([long_data_spt, temp_df], axis=0)

for sub_id, row in HR_response_SLEEP.iterrows():
    for condition in ['high', 'medium', 'low']:
        hr_responses = row[condition]
        time_points = np.arange(len(hr_responses))  # Assuming time points are sequential
        temp_df = pd.DataFrame({
            "sub_ID": [sub_id] * len(hr_responses),
            "movement_intensity": [condition] * len(hr_responses),
            "time": time_points,
            "HR_response": hr_responses
        })
        long_data_sleep = pd.concat([long_data_sleep, temp_df], axis=0)

for sub_id, row in HR_response_WAKE.dropna(subset = ["medium"]).iterrows():
    for condition in ['high', 'medium', 'low']:
        hr_responses = row[condition]
        time_points = np.arange(len(hr_responses))  # Assuming time points are sequential
        temp_df = pd.DataFrame({
            "sub_ID": [sub_id] * len(hr_responses),
            "movement_intensity": [condition] * len(hr_responses),
            "time": time_points,
            "HR_response": hr_responses
        })
        long_data_wake = pd.concat([long_data_wake, temp_df], axis=0)

# Convert subject ID to categorical
long_data_spt['sub_ID'] = long_data_spt['sub_ID'].astype('category')
long_data_sleep['sub_ID'] = long_data_sleep['sub_ID'].astype('category')
long_data_wake['sub_ID'] = long_data_wake['sub_ID'].astype('category')

# Perform repeated-measures ANOVA
aov_spt = AnovaRM(long_data_spt, 'HR_response', 'sub_ID', within=['movement_intensity', 'time']).fit()
aov_sleep = AnovaRM(long_data_sleep, 'HR_response', 'sub_ID', within=['movement_intensity', 'time']).fit()
aov_wake = AnovaRM(long_data_wake, 'HR_response', 'sub_ID', within=['movement_intensity', 'time']).fit()

# Display the ANOVA results

# Check if interaction is significant
interaction_p_value_spt = aov_spt.anova_table.loc['movement_intensity:time', 'Pr > F']
print("Interaction p-value SPT:", interaction_p_value_spt)

interaction_p_value_sleep = aov_sleep.anova_table.loc['movement_intensity:time', 'Pr > F']
print("Interaction p-value SLEEP:", interaction_p_value_sleep)

interaction_p_value_wake = aov_wake.anova_table.loc['movement_intensity:time', 'Pr > F']
print("Interaction p-value WAKE:", interaction_p_value_wake)

Interaction p-value SPT: 1.0774759881427722e-257
Interaction p-value SLEEP: 1.6592495861733402e-138
Interaction p-value WAKE: 2.2304090106642914e-13


### Post hoc second-wise t-test

In [150]:
p_values_high_vs_med_spt = []
p_values_high_vs_med_sleep = []
p_values_high_vs_med_wake = []

p_values_low_vs_med_spt = []
p_values_low_vs_med_sleep = []
p_values_low_vs_med_wake = []

# ACC
p_values_high_vs_med_spt_ACC = []
p_values_high_vs_med_sleep_ACC = []
p_values_high_vs_med_wake_ACC = []

p_values_low_vs_med_spt_ACC = []
p_values_low_vs_med_sleep_ACC = []
p_values_low_vs_med_wake_ACC = []

HR_response_WAKE.dropna(subset = ["medium"], inplace = True)
ACC_response_WAKE.dropna(subset = ["medium"], inplace = True)

for i in range(69):
    high_spt_i = HR_response_SPT["high"].dropna().apply(lambda x: x[i]).values
    med_spt_i = HR_response_SPT["medium"].dropna().apply(lambda x: x[i]).values
    _, p_value_high_vs_med_spt = ttest_rel(high_spt_i, med_spt_i, nan_policy='omit')
    p_values_high_vs_med_spt.append(p_value_high_vs_med_spt)
    high_sleep_i = HR_response_SLEEP["high"].dropna().apply(lambda x: x[i]).values
    med_sleep_i = HR_response_SLEEP["medium"].dropna().apply(lambda x: x[i]).values
    _, p_value_high_vs_med_sleep = ttest_rel(high_sleep_i, med_sleep_i, nan_policy='omit')
    p_values_high_vs_med_sleep.append(p_value_high_vs_med_sleep)
    high_wake_i = HR_response_WAKE["high"].dropna().apply(lambda x: x[i]).values
    med_wake_i = HR_response_WAKE["medium"].dropna().apply(lambda x: x[i]).values
    _, p_value_high_vs_med_wake = ttest_rel(high_wake_i, med_wake_i, nan_policy='omit')
    p_values_high_vs_med_wake.append(p_value_high_vs_med_wake)

    low_spt_i = HR_response_SPT["low"].dropna().apply(lambda x: x[i]).values
    _, p_value_low_vs_med_spt = ttest_rel(low_spt_i, med_spt_i, nan_policy='omit')
    p_values_low_vs_med_spt.append(p_value_low_vs_med_spt)
    low_sleep_i = HR_response_SLEEP["low"].dropna().apply(lambda x: x[i]).values
    _, p_value_low_vs_med_sleep = ttest_rel(low_sleep_i, med_sleep_i, nan_policy='omit')
    p_values_low_vs_med_sleep.append(p_value_low_vs_med_sleep)
    low_wake_i = HR_response_WAKE["low"].dropna().apply(lambda x: x[i]).values
    _, p_value_low_vs_med_wake = ttest_rel(low_wake_i, med_wake_i, nan_policy='omit')
    p_values_low_vs_med_wake.append(p_value_low_vs_med_wake)

    # ACC
    high_spt_i_ACC = ACC_response_SPT["high"].dropna().apply(lambda x: x[i]).values
    med_spt_i_ACC = ACC_response_SPT["medium"].dropna().apply(lambda x: x[i]).values
    _, p_value_high_vs_med_spt_ACC = ttest_rel(high_spt_i_ACC, med_spt_i_ACC, nan_policy='omit')
    p_values_high_vs_med_spt_ACC.append(p_value_high_vs_med_spt_ACC)
    high_sleep_i_ACC = ACC_response_SLEEP["high"].dropna().apply(lambda x: x[i]).values
    med_sleep_i_ACC = ACC_response_SLEEP["medium"].dropna().apply(lambda x: x[i]).values
    _, p_value_high_vs_med_sleep_ACC = ttest_rel(high_sleep_i_ACC, med_sleep_i_ACC, nan_policy='omit')
    p_values_high_vs_med_sleep_ACC.append(p_value_high_vs_med_sleep_ACC)
    high_wake_i_ACC = ACC_response_WAKE["high"].dropna().apply(lambda x: x[i]).values
    med_wake_i_ACC = ACC_response_WAKE["medium"].dropna().apply(lambda x: x[i]).values
    _, p_value_high_vs_med_wake_ACC = ttest_rel(high_wake_i_ACC, med_wake_i_ACC, nan_policy='omit')
    p_values_high_vs_med_wake_ACC.append(p_value_high_vs_med_wake_ACC)

    low_spt_i_ACC = ACC_response_SPT["low"].dropna().apply(lambda x: x[i]).values
    _, p_value_low_vs_med_spt_ACC = ttest_rel(low_spt_i_ACC, med_spt_i_ACC, nan_policy='omit')
    p_values_low_vs_med_spt_ACC.append(p_value_low_vs_med_spt_ACC)
    low_sleep_i_ACC = ACC_response_SLEEP["low"].dropna().apply(lambda x: x[i]).values
    _, p_value_low_vs_med_sleep_ACC = ttest_rel(low_sleep_i_ACC, med_sleep_i_ACC, nan_policy='omit')
    p_values_low_vs_med_sleep_ACC.append(p_value_low_vs_med_sleep_ACC)
    low_wake_i_ACC = ACC_response_WAKE["low"].dropna().apply(lambda x: x[i]).values
    _, p_value_low_vs_med_wake_ACC = ttest_rel(low_wake_i_ACC, med_wake_i_ACC, nan_policy='omit')
    p_values_low_vs_med_wake_ACC.append(p_value_low_vs_med_wake_ACC)

p_values_high_vs_med_spt = np.array(p_values_high_vs_med_spt)
p_values_high_vs_med_sleep = np.array(p_values_high_vs_med_sleep)
p_values_high_vs_med_wake = np.array(p_values_high_vs_med_wake)

p_values_low_vs_med_spt = np.array(p_values_low_vs_med_spt)
p_values_low_vs_med_sleep = np.array(p_values_low_vs_med_sleep)
p_values_low_vs_med_wake = np.array(p_values_low_vs_med_wake)

p_values_high_vs_med_spt_ACC = np.array(p_values_high_vs_med_spt_ACC)
p_values_high_vs_med_sleep_ACC = np.array(p_values_high_vs_med_sleep_ACC)
p_values_high_vs_med_wake_ACC = np.array(p_values_high_vs_med_wake_ACC)

p_values_low_vs_med_spt_ACC = np.array(p_values_low_vs_med_spt_ACC)
p_values_low_vs_med_sleep_ACC = np.array(p_values_low_vs_med_sleep_ACC)
p_values_low_vs_med_wake_ACC = np.array(p_values_low_vs_med_wake_ACC)

In [235]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 10), gridspec_kw={'height_ratios': [4, 1.2]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_SPT["high"].mean(), yerr = sem_high_SPT, fmt = '-o', capsize=3, elinewidth=1.5, label = "High", color = colors[0])
ax1.errorbar(t, HR_response_SPT["medium"].mean(), yerr = sem_medium_SPT, fmt = '-o', capsize=3, elinewidth=1.5, label = "Medium", color = colors[1])
ax1.errorbar(t, HR_response_SPT["low"].mean(), yerr = sem_low_SPT, fmt = '-o', capsize=3, elinewidth=1.5, label = "Low", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-15, 40)
ax1.set_ylim(-8, 53)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
# SET sceintific notation for y-axis
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
# ax1.set_title("dSPT", fontsize = 24)

# Bottom plot: p-values
# ax2.bar(t, p_values_high_vs_med_spt, color = "gray", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_high_vs_med_spt, '-o', color = colors[0], linewidth=1.2, alpha = 0.8, label = "High vs Medium")
# ax2.bar(t, p_values_low_vs_med_spt, color = "black", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_low_vs_med_spt, '-o', linewidth=1.2, color = colors[2], alpha = 0.8, label = "Low vs Medium")
ax2.spines['top'].set_visible(False)

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.0001, 0.01, 1])
ax2.set_yticklabels([0.0001, 0.01, 1], fontsize=16)
ax2.set_ylabel('p', fontsize = 21)
ax2.set_xticks(t[9::10])
ax2.set_xticklabels(t[9::10], fontsize=20)
ax2.set_xlabel('Time (seconds)', fontsize = 21)

ax2.set_xlim(-15, 40)
ax2.set_ylim(0.0000001, 1.8)

ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.0001, color='grey', linestyle='--', linewidth=1.2)

# ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=16, loc = "lower left")

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_tertiles_SPT.png", dpi = 300, bbox_inches = 'tight')

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 10), gridspec_kw={'height_ratios': [4, 1.2]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_SLEEP["high"].mean(), yerr = sem_high_SLEEP, fmt = '-o', capsize=3, elinewidth=1.5, label = "Large full-body movement", color = colors[0])
ax1.errorbar(t, HR_response_SLEEP["medium"].mean(), yerr = sem_medium_SLEEP, fmt = '-o', capsize=3, elinewidth=1.5, label = "Medium full-body movement", color = colors[1])
ax1.errorbar(t, HR_response_SLEEP["low"].mean(), yerr = sem_low_SLEEP, fmt = '-o', capsize=3, elinewidth=1.5, label = "Small full-body movement", color = colors[2])

ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-15, 40)
ax1.set_ylim(-8, 53)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
# ax1.set_title("Sleep", fontsize = 24)

# Bottom plot: p-values
# ax2.bar(t, p_values_high_vs_med_sleep, color = "gray", width=0.5, alpha = 0.8)

ax2.plot(t, p_values_high_vs_med_sleep, '-o', color = colors[0], linewidth=1.2, alpha = 0.8, label = "Large vs Medium")
# ax2.bar(t, p_values_low_vs_med_sleep, color = "black", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_low_vs_med_sleep, '-o', linewidth=1.2, color = colors[2], alpha = 0.8, label = "Small vs Medium")
ax2.spines['top'].set_visible(False)

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.0001, 0.01, 1])
ax2.set_yticklabels([0.0001, 0.01, 1], fontsize=16)
ax2.set_ylabel('p', fontsize = 21)
ax2.set_xticks(t[9::10])
ax2.set_xticklabels(t[9::10], fontsize=20)
ax2.set_xlabel('Time (seconds)', fontsize = 21)

ax2.set_xlim(-15, 40)
ax2.set_ylim(0.0000001, 1.8)

ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.0001, color='grey', linestyle='--', linewidth=1.2)

ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=14, loc = "lower right")

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_tertiles_SLEEP.png", dpi = 300, bbox_inches = 'tight')

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 10), gridspec_kw={'height_ratios': [4, 1.2]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_WAKE["high"].mean(), yerr = sem_high_WAKE, fmt = '-o', capsize=3, elinewidth=1.5, label = "High", color = colors[0])
ax1.errorbar(t, HR_response_WAKE["medium"].mean(), yerr = sem_medium_WAKE, fmt = '-o', capsize=3, elinewidth=1.5, label = "Medium", color = colors[1])
ax1.errorbar(t, HR_response_WAKE["low"].mean(), yerr = sem_low_WAKE, fmt = '-o', capsize=3, elinewidth=1.5, label = "Low", color = colors[2])

ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-15, 40)
ax1.set_ylim(-8, 53)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_tick_params(labelsize=21)
ax1.set_xticks([])
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
# ax1.set_title("Wake", fontsize = 24)

# Bottom plot: p-values
# ax2.bar(t, p_values_high_vs_med_wake, color = "gray", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_high_vs_med_wake, '-o', color = colors[0], linewidth=1.2, alpha = 0.8, label = "High vs Medium")
# ax2.bar(t, p_values_low_vs_med_wake, color = "black", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_low_vs_med_wake, '-o', linewidth=1.2, color = colors[2], alpha = 0.8, label = "Low vs Medium")
ax2.spines['top'].set_visible(False)

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.0001, 0.01, 1])
ax2.set_yticklabels([0.0001, 0.01, 1], fontsize=16)
ax2.set_ylabel('p', fontsize = 21)
ax2.set_xticks(t[9::10])
ax2.set_xticklabels(t[9::10], fontsize=20)
ax2.set_xlabel('Time (seconds)', fontsize = 21)

ax2.set_xlim(-15, 40)
ax2.set_ylim(0.0000001, 1.8)

ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.0001, color='grey', linestyle='--', linewidth=1.2)

# ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=16, loc = "lower left")

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/HR_tertiles_WAKE.png", dpi = 300, bbox_inches = 'tight')

### Acc tertiles

In [157]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 9), gridspec_kw={'height_ratios': [4, 1.2]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t-1, ACC_response_SPT["high"].mean(), yerr = sem_high_ACC_SPT, fmt = '-o', capsize=3, elinewidth=1.5, label = "High", color = colors[0])
ax1.errorbar(t-1, ACC_response_SPT["medium"].mean(), yerr = sem_medium_ACC_SPT, fmt = '-o', capsize=3, elinewidth=1.5, label = "Medium", color = colors[1])
ax1.errorbar(t-1, ACC_response_SPT["low"].mean(), yerr = sem_low_ACC_SPT, fmt = '-o', capsize=3, elinewidth=1.5, label = "Low", color = colors[2])
ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-15, 40)
# ax1.set_ylim(-8, 53)
ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
# set scientific notation for y-ax1is (like 1e6)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
ax1.yaxis.set_tick_params(labelsize=21)
# ax1.set_xticks([])
ax1.set_xlabel("Time (seconds)")
ax1.set_xticks(t[9::10])
ax1.set_xticklabels(t[9::10], fontsize=20)
ax1.set_ylim(ax1.get_ylim()[0], 3.35)
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
ax1.set_title("dSPT", fontsize = 24)

# Bottom plot: p-values
# ax2.bar(t, p_values_high_vs_med_spt_ACC, color = "gray", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_high_vs_med_spt_ACC, '-o', color = colors[0], linewidth=1.2, alpha = 0.8, label = "High vs Medium")
# ax2.bar(t, p_values_low_vs_med_spt_ACC, color = "black", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_low_vs_med_spt_ACC, '-o', linewidth=1.2, color = colors[2], alpha = 0.8, label = "Low vs Medium")
ax2.spines['top'].set_visible(False)

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.0001, 0.01, 1])
ax2.set_yticklabels([0.0001, 0.01, 1], fontsize=16)
ax2.set_ylabel('p', fontsize = 21)
ax2.set_xticks(t[9::10])
ax2.set_xticklabels(t[9::10], fontsize=20)
ax2.set_xlabel('Time (seconds)', fontsize = 21)

ax2.set_xlim(-15, 40)
ax2.set_ylim(0.0000001, 1.8)

ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.0001, color='grey', linestyle='--', linewidth=1.2)



plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACtertiles_C_SPT.png", dpi = 300, bbox_inches = 'tight')

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 9), gridspec_kw={'height_ratios': [4, 1.2]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t-1, ACC_response_SLEEP["high"].mean(), yerr = sem_high_ACC_SLEEP, fmt = '-o', capsize=3, elinewidth=1.5, label = "Large full-body movement", color = colors[0])
ax1.errorbar(t-1, ACC_response_SLEEP["medium"].mean(), yerr = sem_medium_ACC_SLEEP, fmt = '-o', capsize=3, elinewidth=1.5, label = "Medium full-body movement", color = colors[1])
ax1.errorbar(t-1, ACC_response_SLEEP["low"].mean(), yerr = sem_low_ACC_SLEEP, fmt = '-o', capsize=3, elinewidth=1.5, label = "Small full-body movement", color = colors[2])

ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-15, 40)
# ax1.set_ylim(-8, 53)

ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
ax1.yaxis.set_tick_params(labelsize=21)
# ax1.set_xticks([])
ax1.set_xlabel("Time (seconds)")
ax1.set_xticks(t[9::10])
ax1.set_xticklabels(t[9::10], fontsize=20)
ax1.set_ylim(ax1.get_ylim()[0], 3.35)
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)

ax1.set_title("Sleep", fontsize = 24)

# Bottom plot: p-values
# ax2.bar(t, p_values_high_vs_med_spt_ACC, color = "gray", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_high_vs_med_sleep_ACC, '-o', color = colors[0], linewidth=1.2, alpha = 0.8, label = "Large vs Medium")
# ax2.bar(t, p_values_low_vs_med_spt_ACC, color = "black", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_low_vs_med_sleep_ACC, '-o', linewidth=1.2, color = colors[2], alpha = 0.8, label = "Small vs Medium")
ax2.spines['top'].set_visible(False)

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.0001, 0.01, 1])
ax2.set_yticklabels([0.0001, 0.01, 1], fontsize=16)
ax2.set_ylabel('p', fontsize = 21)
ax2.set_xticks(t[9::10])
ax2.set_xticklabels(t[9::10], fontsize=20)
ax2.set_xlabel('Time (seconds)', fontsize = 21)

ax2.set_xlim(-15, 40)
ax2.set_ylim(0.0000001, 1.8)

ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.0001, color='grey', linestyle='--', linewidth=1.2)

ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=14, loc = "lower right")

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACC_tertiles_SLEEP.png", dpi = 300, bbox_inches = 'tight')

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 9), gridspec_kw={'height_ratios': [4, 1.2]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t-1, ACC_response_WAKE["high"].mean(), yerr = sem_high_ACC_WAKE, fmt = '-o', capsize=3, elinewidth=1.5, label = "High", color = colors[0])
ax1.errorbar(t-1, ACC_response_WAKE["medium"].dropna().mean(), yerr = sem_medium_ACC_WAKE, fmt = '-o', capsize=3, elinewidth=1.5, label = "Medium", color = colors[1])
ax1.errorbar(t-1, ACC_response_WAKE["low"].mean(), yerr = sem_low_ACC_WAKE, fmt = '-o', capsize=3, elinewidth=1.5, label = "Low", color = colors[2])

ax1.axvline(x=0, color='black', linestyle='--', linewidth=2.1)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=2.1)

ax1.set_xlim(-15, 40)
# ax1.set_ylim(-8, 53)

ax1.set_ylabel('Movement change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))
ax1.yaxis.set_tick_params(labelsize=21)
# ax1.set_xticks([])
ax1.set_xlabel("Time (seconds)")
ax1.set_xticks(t[9::10])
ax1.set_xticklabels(t[9::10], fontsize=20)
ax1.set_ylim(ax1.get_ylim()[0], 3.35)
# ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)

ax1.set_title("Wake", fontsize = 24)

# Bottom plot: p-values
# ax2.bar(t, p_values_high_vs_med_spt_ACC, color = "gray", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_high_vs_med_wake_ACC, '-o', color = colors[0], linewidth=1.2, alpha = 0.8, label = "High vs Medium")
# ax2.bar(t, p_values_low_vs_med_spt_ACC, color = "black", width=0.5, alpha = 0.8)
ax2.plot(t, p_values_low_vs_med_wake_ACC, '-o', linewidth=1.2, color = colors[2], alpha = 0.8, label = "Low vs Medium")
ax2.spines['top'].set_visible(False)

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=2.1)
ax2.yaxis.set_tick_params(labelsize=18)
ax2.set_yscale('log')
ax2.set_yticks([0.0001, 0.01, 1])
ax2.set_yticklabels([0.0001, 0.01, 1], fontsize=16)
ax2.set_ylabel('p', fontsize = 21)
ax2.set_xticks(t[9::10])
ax2.set_xticklabels(t[9::10], fontsize=20)
ax2.set_xlabel('Time (seconds)', fontsize = 21)

ax2.set_xlim(-15, 40)
ax2.set_ylim(0.0000001, 1.8)

ax2.axhline(y=0.01, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0.0001, color='grey', linestyle='--', linewidth=1.2)

plt.tight_layout(h_pad=0.5)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/ACCtertiles__WAKE.png", dpi = 300, bbox_inches = 'tight')

In [203]:
ACC_response_WAKE["medium"].dropna().mean()

array([-1.48985191e-03, -9.36063982e-03, -5.37911250e-04, -9.96293472e-04,
       -3.52852088e-03, -8.05029628e-03,  1.32281848e-02,  3.23076923e-02,
       -5.90061408e-03, -5.87283816e-03, -1.12887632e-02, -1.33105501e-03,
       -3.67155236e-03,  7.93582143e-06, -2.55119129e-03,  4.08420341e-03,
        1.20464325e-02,  2.21901184e-02,  1.55221235e-02,  5.93221785e-02,
        2.38999192e-01,  1.50843518e+00,  6.03858331e+00,  1.03123964e+01,
        1.33607312e+01,  1.33075742e+01,  1.39201743e+01,  1.31156561e+01,
        1.53067994e+01,  1.31940762e+01,  1.02675844e+01,  9.05159910e+00,
        7.78642934e+00,  7.19165725e+00,  6.67007110e+00,  4.19450812e+00,
        3.55812664e+00,  3.47830241e+00,  2.47522538e+00,  1.81644771e+00,
        1.51536996e+00,  2.25276360e+00,  1.51909987e+00,  6.99016966e-01,
        6.92482954e-01,  7.56940928e-01,  7.76505225e-01,  6.63159180e-01,
        4.82856733e-01,  4.66643058e-01,  5.22122231e-01,  5.57196051e-01,
        4.06486034e-01,  

# Overall movement occurrence rate 

In [8]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects}
twt = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 3600
    twt[sub] = (SIB[sub]["awake.duration"].sum()).total_seconds() / 3600

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

In [9]:
bursts_df["Duration"] = bursts_df["End"] - bursts_df["Start"]
bursts_df["Duration"] = bursts_df["Duration"].apply(lambda x: x.total_seconds())
bursts_df["n_limbs"] = bursts_df["Limbs"].apply(lambda x: len(x))
bursts_df["Category"] = "placeholder"
bursts_df["Laterality"] = "Bilateral"
bursts_df.loc[bursts_df["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df.loc[(bursts_df["Limbs"] == {"LL", "RL"}) | (bursts_df["Limbs"] == {"LL", "T"}) | (bursts_df["Limbs"] == {"RL", "T"}) | (bursts_df["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df.loc[(bursts_df["Limbs"] == {"LW", "RW"}) | (bursts_df["Limbs"] == {"LW", "T"}) | (bursts_df["Limbs"] == {"RW", "T"}) | (bursts_df["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df.loc[(bursts_df["n_limbs"] >= 2) & (bursts_df["n_limbs"] != 5) & (bursts_df["Limbs"] != {"LL", "RL"}) & (bursts_df["Limbs"] != {"LL", "RL", "T"}) & (bursts_df["Limbs"] != {"LW", "RW"}) & (bursts_df["Limbs"] != {"LW", "RW", "T"}) & (bursts_df["Limbs"] != {"LL", "T"}) & (bursts_df["Limbs"] != {"RL", "T"}) & (bursts_df["Limbs"] != {"LW", "T"}) & (bursts_df["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df.loc[bursts_df["n_limbs"] == 1, "Category"] = "Focal"
bursts_df.loc[(bursts_df["Limbs"] == {"LL", "LW"}) | (bursts_df["Limbs"] == {"RL", "RW"}) | (bursts_df["Limbs"] == {"LL", "LW", "T"}) | (bursts_df["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_df.loc[(bursts_df["Limbs"] == {"LL", "T"}) | (bursts_df["Limbs"] == {"LW", "T"}) | (bursts_df["Limbs"] == {"RL", "T"}) | (bursts_df["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_df.loc[(bursts_df["Limbs"] == {"LL", "RL"}) | (bursts_df["Limbs"] == {"LW"}) | (bursts_df["Limbs"] == {"RW"}) | (bursts_df["Limbs"] == {"LL"}) | (bursts_df["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_df.loc[bursts_df["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_df_focal = bursts_df[bursts_df["Category"] == "Focal"].reset_index(drop=True)
bursts_df_focal["Focal limb"] = "placeholder"
bursts_df_focal.loc[(bursts_df_focal["Limbs"] == {"LL"}), "Focal limb"] = "Left ankle"
bursts_df_focal.loc[(bursts_df_focal["Limbs"] == {"RL"}), "Focal limb"] = "Right ankle"
bursts_df_focal.loc[(bursts_df_focal["Limbs"] == {"LW"}), "Focal limb"] = "Left wrist"
bursts_df_focal.loc[(bursts_df_focal["Limbs"] == {"RW"}), "Focal limb"] = "Right wrist"
bursts_df_focal.loc[(bursts_df_focal["Limbs"] == {"T"}), "Focal limb"] = "Trunk"

bursts_df_focal_sleep = bursts_df_focal[bursts_df_focal["SIB"] == 1].reset_index(drop=True)
bursts_df_focal_wake = bursts_df_focal[bursts_df_focal["SIB"] == 0].reset_index(drop=True)

bursts_df_sleep = bursts_df[bursts_df["SIB"] == 1].reset_index(drop=True)
# bursts_df_sleep["Duration"] = bursts_df_sleep["End"] - bursts_df_sleep["Start"]
# bursts_df_sleep["Duration"] = bursts_df_sleep["Duration"].apply(lambda x: x.total_seconds())
# bursts_df_sleep["n_limbs"] = bursts_df_sleep["Limbs"].apply(lambda x: len(x))
# bursts_df_sleep["Category"] = "placeholder"
# bursts_df_sleep["Laterality"] = "Bilateral"
# bursts_df_sleep.loc[bursts_df_sleep["n_limbs"] == 5, "Category"] = "Full Body"
# bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_df_sleep["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
# bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RW", "T"}) | (bursts_df_sleep["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
# bursts_df_sleep.loc[(bursts_df_sleep["n_limbs"] >= 2) & (bursts_df_sleep["n_limbs"] != 5) & (bursts_df_sleep["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep["Limbs"] != {"LL", "T"}) & (bursts_df_sleep["Limbs"] != {"RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "T"}) & (bursts_df_sleep["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
# bursts_df_sleep.loc[bursts_df_sleep["n_limbs"] == 1, "Category"] = "Focal"
# bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "LW"}) | (bursts_df_sleep["Limbs"] == {"RL", "RW"}) | (bursts_df_sleep["Limbs"] == {"LL", "LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
# bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_df_sleep["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
# bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep["Limbs"] == {"LW"}) | (bursts_df_sleep["Limbs"] == {"RW"}) | (bursts_df_sleep["Limbs"] == {"LL"}) | (bursts_df_sleep["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
# bursts_df_sleep.loc[bursts_df_sleep["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_df_wake = bursts_df[bursts_df["SIB"] == 0].reset_index(drop=True)
# bursts_df_wake["Duration"] = bursts_df_wake["End"] - bursts_df_wake["Start"]
# bursts_df_wake["Duration"] = bursts_df_wake["Duration"].apply(lambda x: x.total_seconds())
# bursts_df_wake["n_limbs"] = bursts_df_wake["Limbs"].apply(lambda x: len(x))
# bursts_df_wake["Category"] = "placeholder"
# bursts_df_wake["Laterality"] = "Bilateral"
# bursts_df_wake.loc[bursts_df_wake["n_limbs"] == 5, "Category"] = "Full Body"
# bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_df_wake["Limbs"] == {"LL", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "T"}) | (bursts_df_wake["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
# bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LW", "RW"}) | (bursts_df_wake["Limbs"] == {"LW", "T"}) | (bursts_df_wake["Limbs"] == {"RW", "T"}) | (bursts_df_wake["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
# bursts_df_wake.loc[(bursts_df_wake["n_limbs"] >= 2) & (bursts_df_wake["n_limbs"] != 5) & (bursts_df_wake["Limbs"] != {"LL", "RL"}) & (bursts_df_wake["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_wake["Limbs"] != {"LW", "RW"}) & (bursts_df_wake["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_wake["Limbs"] != {"LL", "T"}) & (bursts_df_wake["Limbs"] != {"RL", "T"}) & (bursts_df_wake["Limbs"] != {"LW", "T"}) & (bursts_df_wake["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
# bursts_df_wake.loc[bursts_df_wake["n_limbs"] == 1, "Category"] = "Focal"
# bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "LW"}) | (bursts_df_wake["Limbs"] == {"RL", "RW"}) | (bursts_df_wake["Limbs"] == {"LL", "LW", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
# bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "T"}) | (bursts_df_wake["Limbs"] == {"LW", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "T"}) | (bursts_df_wake["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
# bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_df_wake["Limbs"] == {"LW"}) | (bursts_df_wake["Limbs"] == {"RW"}) | (bursts_df_wake["Limbs"] == {"LL"}) | (bursts_df_wake["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
# bursts_df_wake.loc[bursts_df_wake["Limbs"] == {"T"}, "Laterality"] = "None"

tst_df = pd.DataFrame(tst, index = ["TST"]).T.sort_index()
twt_df = pd.DataFrame(twt, index = ["TWT"]).T.sort_index()

In [10]:
spt_df = pd.DataFrame([tst_df["TST"], twt_df["TWT"]], index = ["TST", "TWT"]).T
spt_dur_df = spt_df.sum(axis = 1)
df_index = bursts_df_wake.groupby(["sub_ID"])["Duration"].describe()["count"].T.div(twt_df["TWT"], axis = 0).describe()
print(df_index.loc["mean"], df_index.loc["std"])

df_duration_sleep = bursts_df_wake.groupby(["sub_ID"])["Duration"].mean()
df_duration_sleep.mean(), df_duration_sleep.std()

# df_index_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
# df_index_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
# for movement in df_index_sleep.columns:
#     mean = df_index.loc["mean", movement].round(1)
#     std = df_index.loc["std", movement].round(1)
#     perc75 = df_index.loc["75%", movement].round(1)

90.35908783464289 21.42993882981924


(16.882710335580537, 6.945090231045331)

In [13]:
# create a df where, for each movement, there is the mean (std): for example for Full Body: 0.5 (0.2)
df_index = bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_index_sleep.columns:
    mean = df_index.loc["mean", movement].round(1)
    std = df_index.loc["std", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_index_sleep = df_index_sleep.T
df_index_sleep.columns = ["Index (n/h)"]

df_duration = bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_duration_sleep.columns:
    mean = df_duration.loc["mean", movement].round(1)
    std = df_duration.loc["std", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_duration_sleep = df_duration_sleep.T
df_duration_sleep.columns = ["Duration (s)"]

df_p2p = (bursts_df_sleep.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_p2p_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_p2p_sleep.columns:
    mean = df_p2p.loc["mean", movement].round(2)
    std = df_p2p.loc["std", movement].round(2)
    perc75 = df_p2p.loc["75%", movement].round(2)
    df_p2p_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_p2p_sleep = df_p2p_sleep.T
df_p2p_sleep.columns = ["p2p (g)"]

df_AUC = (bursts_df_sleep.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_AUC_sleep.columns:
    mean = df_AUC.loc["mean", movement].round(1)
    std = df_AUC.loc["std", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_sleep = df_AUC_sleep.T
df_AUC_sleep.columns = ["AUC (g·s)"]

table_movements_sleep = pd.concat([df_index_sleep, df_duration_sleep, df_p2p_sleep, df_AUC_sleep], axis = 1)



table_laterality_sleep = pd.concat([df_index_laterality_sleep, df_duration_laterality_sleep, df_p2p_laterality_sleep, df_AUC_laterality_sleep], axis = 1)

table_sleep = pd.concat([table_movements_sleep, table_laterality_sleep], axis = 0)

NameError: name 'df_index_laterality_sleep' is not defined

In [34]:
table_sleep

,Index (n/h),Duration (s),p2p (g),AUC (g·s)
Focal,9.0 (3.6),1.4 (0.3),0.04 (0.01),4.8 (2.2)
Full Body,5.2 (3.1),10.3 (4.0),1.21 (0.48),400.1 (173.6)
Lower Body,3.2 (1.8),3.5 (1.1),0.11 (0.04),20.3 (11.5)
Upper Body,1.0 (0.7),3.6 (1.2),0.21 (0.2),42.1 (46.7)
Mixed,3.0 (1.6),5.5 (2.5),0.27 (0.09),58.0 (25.4)
Unilateral,10.9 (4.5),1.9 (0.4),0.05 (0.01),7.7 (3.4)
Bilateral,9.8 (4.0),7.8 (3.0),0.77 (0.33),238.3 (116.6)


In [35]:
# WAKE
# create a df where, for each movement, there is the mean (std): for example for Full Body: 0.5 (0.2)
df_index = bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(twt_df["TWT"], axis = 0).describe()
df_index_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_index_wake.columns:
    mean = df_index.loc["mean", movement].round(1)
    std = df_index.loc["std", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_wake[movement] = str(mean) + " (" + str(std) + ")"
df_index_wake = df_index_wake.T
df_index_wake.columns = ["Index (n/h)"]

df_duration = bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_duration_wake.columns:
    mean = df_duration.loc["mean", movement].round(1)
    std = df_duration.loc["std", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_wake[movement] = str(mean) + " (" + str(std) + ")"
df_duration_wake = df_duration_wake.T
df_duration_wake.columns = ["Duration (s)"]

df_p2p = (bursts_df_wake.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_p2p_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_p2p_wake.columns:
    mean = df_p2p.loc["mean", movement].round(2)
    std = df_p2p.loc["std", movement].round(2)
    perc75 = df_p2p.loc["75%", movement].round(2)
    df_p2p_wake[movement] = str(mean) + " (" + str(std) + ")"
df_p2p_wake = df_p2p_wake.T
df_p2p_wake.columns = ["p2p (g)"]

df_AUC = (bursts_df_wake.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_AUC_wake.columns:
    mean = df_AUC.loc["mean", movement].round(1)
    std = df_AUC.loc["std", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_wake[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_wake = df_AUC_wake.T
df_AUC_wake.columns = ["AUC (g·s)"]

table_movements_wake = pd.concat([df_index_wake, df_duration_wake, df_p2p_wake, df_AUC_wake], axis = 1)

df_index_laterality = bursts_df_wake.groupby(["Laterality", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(twt_df["TWT"], axis = 0).describe()
df_index_laterality_wake = pd.DataFrame(["a", "a"]).T
df_index_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_index_laterality_wake.columns:
    mean = df_index_laterality.loc["mean", movement].round(1)
    std = df_index_laterality.loc["std", movement].round(1)
    perc75 = df_index_laterality.loc["75%", movement].round(1)
    df_index_laterality_wake[movement] = str(mean) + " (" + str(std) + ")"
df_index_laterality_wake = df_index_laterality_wake.T
df_index_laterality_wake.columns = ["Index (n/h)"]
df_index_laterality_wake

df_duration_laterality = bursts_df_wake.groupby(["Laterality", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_laterality_wake = pd.DataFrame(["a", "a"]).T
df_duration_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_duration_laterality_wake.columns:
    mean = df_duration_laterality.loc["mean", movement].round(1)
    std = df_duration_laterality.loc["std", movement].round(1)
    perc75 = df_duration_laterality.loc["75%", movement].round(1)
    df_duration_laterality_wake[movement] = str(mean) + " (" + str(std) + ")"
df_duration_laterality_wake = df_duration_laterality_wake.T
df_duration_laterality_wake.columns = ["Duration (s)"]
df_duration_laterality_wake

df_p2p_laterality = (bursts_df_wake.groupby(["Laterality", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_laterality_wake = pd.DataFrame(["a", "a"]).T
df_p2p_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_p2p_laterality_wake.columns:
    mean = df_p2p_laterality.loc["mean", movement].round(2)
    std = df_p2p_laterality.loc["std", movement].round(2)
    perc75 = df_p2p_laterality.loc["75%", movement].round(2)
    df_p2p_laterality_wake[movement] = str(mean) + " (" + str(std) + ")"
df_p2p_laterality_wake = df_p2p_laterality_wake.T
df_p2p_laterality_wake.columns = ["p2p (g)"]
df_p2p_laterality_wake

df_AUC_laterality = (bursts_df_wake.groupby(["Laterality", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_laterality_wake = pd.DataFrame(["a", "a"]).T
df_AUC_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_AUC_laterality_wake.columns:
    mean = df_AUC_laterality.loc["mean", movement].round(1)
    std = df_AUC_laterality.loc["std", movement].round(1)
    perc75 = df_AUC_laterality.loc["75%", movement].round(1)
    df_AUC_laterality_wake[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_laterality_wake = df_AUC_laterality_wake.T
df_AUC_laterality_wake.columns = ["AUC (g·s)"]
df_AUC_laterality_wake

table_laterality_wake = pd.concat([df_index_laterality_wake, df_duration_laterality_wake, df_p2p_laterality_wake, df_AUC_laterality_wake], axis = 1)

table_wake = pd.concat([table_movements_wake, table_laterality_wake], axis = 0)

In [36]:
# spt
spt_df = pd.DataFrame([tst_df["TST"], twt_df["TWT"]], index = ["TST", "TWT"]).T
spt_dur_df = spt_df.sum(axis = 1)
# create a df where, for each movement, there is the mean (std): for example for Full Body: 0.5 (0.2)
df_index = bursts_df.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(spt_dur_df, axis = 0).describe()
df_index_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_spt.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_index_spt.columns:
    mean = df_index.loc["mean", movement].round(1)
    std = df_index.loc["std", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_spt[movement] = str(mean) + " (" + str(std) + ")"
df_index_spt = df_index_spt.T
df_index_spt.columns = ["Index (n/h)"]

df_duration = bursts_df.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_spt.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_duration_spt.columns:
    mean = df_duration.loc["mean", movement].round(1)
    std = df_duration.loc["std", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_spt[movement] = str(mean) + " (" + str(std) + ")"
df_duration_spt = df_duration_spt.T
df_duration_spt.columns = ["Duration (s)"]

df_p2p = (bursts_df.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_p2p_spt.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_p2p_spt.columns:
    mean = df_p2p.loc["mean", movement].round(2)
    std = df_p2p.loc["std", movement].round(2)
    perc75 = df_p2p.loc["75%", movement].round(2)
    df_p2p_spt[movement] = str(mean) + " (" + str(std) + ")"
df_p2p_spt = df_p2p_spt.T
df_p2p_spt.columns = ["p2p (g)"]

df_AUC = (bursts_df.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_spt.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_AUC_spt.columns:
    mean = df_AUC.loc["mean", movement].round(1)
    std = df_AUC.loc["std", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_spt[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_spt = df_AUC_spt.T
df_AUC_spt.columns = ["AUC (g·s)"]

table_movements_spt = pd.concat([df_index_spt, df_duration_spt, df_p2p_spt, df_AUC_spt], axis = 1)

df_index_laterality = bursts_df.groupby(["Laterality", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(spt_dur_df, axis = 0).describe()
df_index_laterality_spt = pd.DataFrame(["a", "a"]).T
df_index_laterality_spt.columns = ["Unilateral", "Bilateral"]
for movement in df_index_laterality_spt.columns:
    mean = df_index_laterality.loc["mean", movement].round(1)
    std = df_index_laterality.loc["std", movement].round(1)
    perc75 = df_index_laterality.loc["75%", movement].round(1)
    df_index_laterality_spt[movement] = str(mean) + " (" + str(std) + ")"
df_index_laterality_spt = df_index_laterality_spt.T
df_index_laterality_spt.columns = ["Index (n/h)"]
df_index_laterality_spt

df_duration_laterality = bursts_df.groupby(["Laterality", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_laterality_spt = pd.DataFrame(["a", "a"]).T
df_duration_laterality_spt.columns = ["Unilateral", "Bilateral"]
for movement in df_duration_laterality_spt.columns:
    mean = df_duration_laterality.loc["mean", movement].round(1)
    std = df_duration_laterality.loc["std", movement].round(1)
    perc75 = df_duration_laterality.loc["75%", movement].round(1)
    df_duration_laterality_spt[movement] = str(mean) + " (" + str(std) + ")"
df_duration_laterality_spt = df_duration_laterality_spt.T
df_duration_laterality_spt.columns = ["Duration (s)"]
df_duration_laterality_spt

df_p2p_laterality = (bursts_df.groupby(["Laterality", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_laterality_spt = pd.DataFrame(["a", "a"]).T
df_p2p_laterality_spt.columns = ["Unilateral", "Bilateral"]
for movement in df_p2p_laterality_spt.columns:
    mean = df_p2p_laterality.loc["mean", movement].round(2)
    std = df_p2p_laterality.loc["std", movement].round(2)
    perc75 = df_p2p_laterality.loc["75%", movement].round(2)
    df_p2p_laterality_spt[movement] = str(mean) + " (" + str(std) + ")"
df_p2p_laterality_spt = df_p2p_laterality_spt.T
df_p2p_laterality_spt.columns = ["p2p (g)"]
df_p2p_laterality_spt

df_AUC_laterality = (bursts_df.groupby(["Laterality", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_laterality_spt = pd.DataFrame(["a", "a"]).T
df_AUC_laterality_spt.columns = ["Unilateral", "Bilateral"]
for movement in df_AUC_laterality_spt.columns:
    mean = df_AUC_laterality.loc["mean", movement].round(1)
    std = df_AUC_laterality.loc["std", movement].round(1)
    perc75 = df_AUC_laterality.loc["75%", movement].round(1)
    df_AUC_laterality_spt[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_laterality_spt = df_AUC_laterality_spt.T
df_AUC_laterality_spt.columns = ["AUC (g·s)"]
df_AUC_laterality_spt

table_laterality_spt = pd.concat([df_index_laterality_spt, df_duration_laterality_spt, df_p2p_laterality_spt, df_AUC_laterality_spt], axis = 1)

table_spt = pd.concat([table_movements_spt, table_laterality_spt], axis = 0)

In [46]:
# Focal sleep
# create a df where, for each movement, there is the mean (std): for example for Full Body: 0.5 (0.2)
df_index = bursts_df_focal_sleep.groupby(["Focal limb", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_focal_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_focal_sleep.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]
for movement in df_index_focal_sleep.columns:
    mean = df_index.loc["mean", movement].round(1)
    std = df_index.loc["std", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_focal_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_index_focal_sleep = df_index_focal_sleep.T
df_index_focal_sleep.columns = ["Index (n/h)"]

df_duration = bursts_df_focal_sleep.groupby(["Focal limb", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_focal_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_focal_sleep.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]
for movement in df_duration_focal_sleep.columns:
    mean = df_duration.loc["mean", movement].round(1)
    std = df_duration.loc["std", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_focal_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_duration_focal_sleep = df_duration_focal_sleep.T
df_duration_focal_sleep.columns = ["Duration (s)"]

df_AUC = (bursts_df_focal_sleep.groupby(["Focal limb", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_focal_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_focal_sleep.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]
for movement in df_AUC_focal_sleep.columns:
    mean = df_AUC.loc["mean", movement].round(1)
    std = df_AUC.loc["std", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_focal_sleep[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_focal_sleep = df_AUC_focal_sleep.T
df_AUC_focal_sleep.columns = ["AUC (g·s)"]

table_movements_focal_sleep = pd.concat([df_index_focal_sleep, df_duration_focal_sleep, df_AUC_focal_sleep], axis = 1)

table_focal_sleep = table_movements_focal_sleep

# Focal wake
# create a df where, for each movement, there is the mean (std): for example for Full Body: 0.5 (0.2)
df_index = bursts_df_focal_wake.groupby(["Focal limb", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(twt_df["TWT"], axis = 0).describe()
df_index_focal_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_focal_wake.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]

for movement in df_index_focal_wake.columns:
    mean = df_index.loc["mean", movement].round(1)
    std = df_index.loc["std", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_focal_wake[movement] = str(mean) + " (" + str(std) + ")"
df_index_focal_wake = df_index_focal_wake.T
df_index_focal_wake.columns = ["Index (n/h)"]

df_duration = bursts_df_focal_wake.groupby(["Focal limb", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_focal_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_focal_wake.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]

for movement in df_duration_focal_wake.columns:
    mean = df_duration.loc["mean", movement].round(1)
    std = df_duration.loc["std", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_focal_wake[movement] = str(mean) + " (" + str(std) + ")"
df_duration_focal_wake = df_duration_focal_wake.T
df_duration_focal_wake.columns = ["Duration (s)"]

df_AUC = (bursts_df_focal_wake.groupby(["Focal limb", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_focal_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_focal_wake.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]

for movement in df_AUC_focal_wake.columns:
    mean = df_AUC.loc["mean", movement].round(1)
    std = df_AUC.loc["std", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_focal_wake[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_focal_wake = df_AUC_focal_wake.T
df_AUC_focal_wake.columns = ["AUC (g·s)"]

table_movements_focal_wake = pd.concat([df_index_focal_wake, df_duration_focal_wake, df_AUC_focal_wake], axis = 1)

table_focal_wake = table_movements_focal_wake

# spt focal
df_index = bursts_df_focal.groupby(["Focal limb", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_focal_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_focal_spt.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]
for movement in df_index_focal_spt.columns:
    mean = df_index.loc["mean", movement].round(1)
    std = df_index.loc["std", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_focal_spt[movement] = str(mean) + " (" + str(std) + ")"
df_index_focal_spt = df_index_focal_spt.T
df_index_focal_spt.columns = ["Index (n/h)"]

df_duration = bursts_df_focal.groupby(["Focal limb", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_focal_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_focal_spt.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]
for movement in df_duration_focal_spt.columns:
    mean = df_duration.loc["mean", movement].round(1)
    std = df_duration.loc["std", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_focal_spt[movement] = str(mean) + " (" + str(std) + ")"
df_duration_focal_spt = df_duration_focal_spt.T
df_duration_focal_spt.columns = ["Duration (s)"]

df_AUC = (bursts_df_focal.groupby(["Focal limb", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_focal_spt = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_focal_spt.columns = ["Left ankle", "Right ankle", "Left wrist", "Right wrist", "Trunk"]
for movement in df_AUC_focal_spt.columns:
    mean = df_AUC.loc["mean", movement].round(1)
    std = df_AUC.loc["std", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_focal_spt[movement] = str(mean) + " (" + str(std) + ")"
df_AUC_focal_spt = df_AUC_focal_spt.T
df_AUC_focal_spt.columns = ["AUC (g·s)"]

table_movements_focal_spt = pd.concat([df_index_focal_spt, df_duration_focal_spt, df_AUC_focal_spt], axis = 1)

table_focal_spt = table_movements_focal_spt



In [47]:
table = pd.concat([table_spt, table_sleep, table_wake], axis = 1)
table_focal = pd.concat([table_focal_spt, table_focal_sleep, table_focal_wake], axis = 1)
table_focal

,Index (n/h),Duration (s),AUC (g·s),Index (n/h),Duration (s),AUC (g·s),Index (n/h),Duration (s),AUC (g·s)
Left ankle,2.3 (0.9),1.5 (0.5),3.8 (1.9),2.0 (0.7),1.5 (0.5),3.8 (1.8),3.3 (3.1),1.3 (0.7),3.9 (3.9)
Right ankle,2.4 (1.3),1.5 (0.5),3.9 (1.7),2.3 (1.2),1.6 (0.5),3.9 (1.7),2.3 (1.8),1.2 (0.7),3.4 (2.8)
Left wrist,2.9 (1.6),1.3 (0.3),4.6 (1.7),2.3 (1.5),1.2 (0.4),4.2 (2.1),6.7 (6.0),1.4 (0.5),4.5 (2.7)
Right wrist,2.8 (1.5),1.7 (1.0),7.5 (4.9),2.3 (1.3),1.5 (0.5),6.9 (4.7),4.6 (2.9),2.3 (1.9),9.6 (8.9)
Trunk,0.9 (0.9),1.2 (0.7),2.4 (1.6),0.9 (0.9),1.3 (0.7),2.5 (1.6),1.8 (0.7),0.9 (0.6),1.7 (1.1)


In [48]:
# table.to_excel("/Users/marcellosicbaldi/Desktop/paper-Sleep/tables/table1.xlsx")
table_focal.to_excel("/Users/marcellosicbaldi/Desktop/paper-Sleep/tables/table_focal.xlsx")

# Movement topography

In [11]:
bursts_df_sleep["PC"].replace(0, np.nan, inplace = True)
bursts_df_sleep["PC"] = bursts_df_sleep["PC"].apply(lambda x: 1 if x < 100 else np.nan)

bursts_df_wake["PC"].replace(0, np.nan, inplace = True)
bursts_df_wake["PC"] = bursts_df_wake["PC"].apply(lambda x: 1 if x < 100 else np.nan)

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_960/2584907591.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bursts_df_sleep["PC"].replace(0, np.nan, inplace = True)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_960/2584907591.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [12]:
# SPT
mov = "Mixed"
n_tot_spt = bursts_df.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
perc_spt_bySub = (bursts_df.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[mov] / n_tot_spt)*100
perc_spt_bySub.mean(), perc_spt_bySub.std()

(12.206837069240827, 4.95587003712065)

In [13]:
# Sleep
mov = "Focal"
n_tot_sleep = bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
perc_sleep_bySub = (bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[mov] / n_tot_sleep)*100
perc_sleep_bySub.mean(), perc_sleep_bySub.std()

(41.932631518352856, 12.614659390021123)

In [14]:
# Wake
mov = "Full Body"
n_tot_wake = bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
perc_wake_bySub = (bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[mov] / n_tot_wake)*100
perc_wake_bySub.mean(), perc_wake_bySub.std()

(66.16435105401492, 8.451629630831588)

#### Sleep vs Wake movement topography

In [28]:
from scipy.stats import shapiro

p_values_shapiro_spt = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    n_tot_sleep = bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
    perc_sleep_bySub = (bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[cat] / n_tot_sleep)*100

    n_tot_wake = bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
    perc_wake_bySub = (bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[cat] / n_tot_wake)*100

    p_values_shapiro_spt[cat] = shapiro(perc_sleep_bySub)[1], shapiro(perc_wake_bySub)[1]

    if p_values_shapiro_spt[cat][0] > 0.05 and p_values_shapiro_spt[cat][1] > 0.05:
        print(f"{cat}: Data is normally distributed in both halves. Use a parametric test.")
    else:
        print(f"{cat}: Data is NOT normally distributed in at least one half. Use a non-parametric test.")

Full Body: Data is normally distributed in both halves. Use a parametric test.
Lower Body: Data is normally distributed in both halves. Use a parametric test.
Upper Body: Data is normally distributed in both halves. Use a parametric test.
Mixed: Data is normally distributed in both halves. Use a parametric test.
Focal: Data is NOT normally distributed in at least one half. Use a non-parametric test.


All normally distributed except focal

In [15]:
from scipy.stats import shapiro
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon

p_values_sleep_vs_wake = {}

for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    n_tot_sleep = bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
    perc_sleep_bySub = (bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[cat] / n_tot_sleep)*100

    n_tot_wake = bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack().sum(axis = 1)
    perc_wake_bySub = (bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack()[cat] / n_tot_wake)*100

    if cat == "Full Body":
        p_values_sleep_vs_wake[cat] = wilcoxon(perc_sleep_bySub, perc_wake_bySub, nan_policy="omit")[1]
    else:
        p_values_sleep_vs_wake[cat] = ttest_rel(perc_sleep_bySub, perc_wake_bySub, nan_policy="omit")[1]

In [19]:
n_tot_fullBody_sleep.sum(), n_tot_fullBody_wake.sum()

(403, 508.0)

In [16]:
n_tot_fullBody_sleep = bursts_df_sleep.groupby(["sub_ID", "Category"])["Duration"].count().unstack()["Full Body"]
(bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"] / n_tot_fullBody_sleep *100).mean(), (bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"] / n_tot_fullBody_sleep *100).std()

n_tot_fullBody_wake = bursts_df_wake.groupby(["sub_ID", "Category"])["Duration"].count().unstack()["Full Body"]
(bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"] / n_tot_fullBody_wake *100).mean(), (bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"] / n_tot_fullBody_wake *100).std()

(34.30459678686359, 18.013903561726444)

In [59]:
pc_sleep = (bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"] / n_tot_fullBody_sleep *100)
pc_wake = (bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"] / n_tot_fullBody_wake *100)

shapiro(pc_sleep)[1], shapiro(pc_wake)[1]

if shapiro(pc_sleep)[1] > 0.05 and shapiro(pc_wake)[1] > 0.05:
    print("Data is normally distributed in both halves. Use a parametric test.")
else:
    print("Data is NOT normally distributed in at least one half. Use a non-parametric test.")

ttest_rel(pc_sleep, pc_wake, nan_policy="omit")[1]

Data is normally distributed in both halves. Use a parametric test.


0.0006190679938064716

In [18]:
# Percentage of full body movements that are posture changes in sleep
n_tot_fullBody_sleep = bursts_df_sleep[bursts_df_sleep["Category"] == "Full Body"].shape[0]
(bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"].sum() / n_tot_fullBody_sleep)*100

#median and IQR
((bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"])).describe(percentiles=[0.25, 0.75, 0.9])

count    12.000000
mean      3.166667
std       3.010084
min       0.000000
25%       1.000000
50%       2.500000
75%       5.000000
90%       7.700000
max       9.000000
Name: Full Body, dtype: float64

In [19]:
# Percentage of full body movements that are posture changes in wake
n_tot_fullBody_wake = bursts_df_wake[bursts_df_wake["Category"] == "Full Body"].shape[0]
(bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"].sum() / n_tot_fullBody_wake)*100

# #median and IQR
((bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"])).describe(percentiles=[0.25, 0.75, 0.9])

count    12.000000
mean     12.000000
std       4.954337
min       6.000000
25%       9.000000
50%      11.000000
75%      14.000000
90%      19.400000
max      22.000000
Name: Full Body, dtype: float64

In [176]:
perc_sleep = pd.DataFrame(bursts_df_sleep.groupby(["Category"])["Duration"].count())# / bursts_df_sleep["Duration"].count())
perc_wake = pd.DataFrame(bursts_df_wake.groupby(["Category"])["Duration"].count()) #/ bursts_df_wake["Duration"].count())
perc_spt = pd.DataFrame(bursts_df.groupby(["Category"])["Duration"].count()) #/ bursts_df["Duration"].count())

df = pd.concat([perc_sleep, perc_wake, perc_spt], axis = 1)
df.columns = ["Sleep", "Wake", "SPT"]

# df.plot(kind = "bar", figsize = (15, 9), stacked = True)

### Figure2

In [180]:
# Sleep
from matplotlib.patches import ConnectionPatch
singleLimb_df = bursts_df_sleep[bursts_df_sleep["Category"] == "Focal"].reset_index(drop=True)
perc_LW = singleLimb_df[singleLimb_df["Limbs"] == {"LW"}].shape[0] / singleLimb_df.shape[0]
perc_RW = singleLimb_df[singleLimb_df["Limbs"] == {"RW"}].shape[0] / singleLimb_df.shape[0]
perc_LL = singleLimb_df[singleLimb_df["Limbs"] == {"LL"}].shape[0] / singleLimb_df.shape[0]
perc_RL = singleLimb_df[singleLimb_df["Limbs"] == {"RL"}].shape[0] / singleLimb_df.shape[0]
perc_T = singleLimb_df[singleLimb_df["Limbs"] == {"T"}].shape[0] / singleLimb_df.shape[0]

perc_LW, perc_RW, perc_RL, perc_LL, perc_T
# do what above but with the sleep data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9), gridspec_kw={'width_ratios': [2, 1]})
fig.subplots_adjust(wspace=0)

# pie chart parameters
overall_ratios_sleep = df["Sleep"].values
labels_sleep = df.index.to_list()
labels_sleep[4] = "\nUpper\nBody"

explode = [0.1, 0, 0, 0, 0]
# rotate so that first wedge is split by the x-axis
angle = 49 * overall_ratios_sleep[0]
wedges, *_ = ax1.pie(overall_ratios_sleep, autopct='%1.1f%%', startangle=angle,
                     labels=labels_sleep, explode=explode, textprops = {"fontsize": 21})

# bar chart parameters
singleLimb_ratios = [perc_LW, perc_RW, perc_RL, perc_LL, perc_T]
singleLimb_labels = ["LW", "RW", "RA", "LA", "T"]
bottom = 1
width = .2

# Adding from the top matches the legend.
for j, (height, label) in enumerate(reversed([*zip(singleLimb_ratios, singleLimb_labels)])):
    bottom -= height
    bc = ax2.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.20 * j)
    ax2.bar_label(bc, labels=[f"{height:.0%}"], label_type='center', fontsize = 21)

ax2.set_title('Focal movements', fontsize = 21)
ax2.legend(frameon = True, fancybox = True, shadow = True, fontsize = 16)
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

# use ConnectionPatch to draw lines between the two plots
theta1, theta2 = wedges[0].theta1, wedges[0].theta2
center, r = wedges[0].center, wedges[0].r
bar_height = sum(singleLimb_ratios)

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = r * np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(-width / 2, bar_height), coordsA=ax2.transData,
                      xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
con.set_linewidth(4)
ax2.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = r * np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax2.transData,
                        xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
ax2.add_artist(con)
con.set_linewidth(4)

# plt.suptitle("Sleep", fontsize = 24)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/pie/movement_distribution_sleep.png", dpi = 300, bbox_inches = 'tight')

In [181]:
# Wake
singleLimb_df = bursts_df_wake[bursts_df_wake["Category"] == "Focal"].reset_index(drop=True)
perc_LW = singleLimb_df[singleLimb_df["Limbs"] == {"LW"}].shape[0] / singleLimb_df.shape[0]
perc_RW = singleLimb_df[singleLimb_df["Limbs"] == {"RW"}].shape[0] / singleLimb_df.shape[0]
perc_LL = singleLimb_df[singleLimb_df["Limbs"] == {"LL"}].shape[0] / singleLimb_df.shape[0]
perc_RL = singleLimb_df[singleLimb_df["Limbs"] == {"RL"}].shape[0] / singleLimb_df.shape[0]
perc_T = singleLimb_df[singleLimb_df["Limbs"] == {"T"}].shape[0] / singleLimb_df.shape[0]
# do what above but with the wake data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9), gridspec_kw={'width_ratios': [2, 1]})
fig.subplots_adjust(wspace=0)

# pie chart parameters
overall_ratios_wake = df["Wake"].values
labels_wake = df.index.to_list()
labels_wake[4] = "\n\n\nUpper\nBody"

explode = [0.1, 0, 0, 0, 0]
# rotate so that first wedge is split by the x-axis
angle = -30
wedges, *_ = ax1.pie(overall_ratios_wake, autopct='%1.1f%%', startangle=angle,
                     labels=labels_wake, explode=explode, textprops = {"fontsize": 21})

# bar chart parameters
singleLimb_ratios = [perc_LW, perc_RW, perc_LL, perc_RL, perc_T]
singleLimb_labels = ["LW", "RW", "LA", "RA", "T"]
bottom = 1
width = .2

# Adding from the top matches the legend.
for j, (height, label) in enumerate(reversed([*zip(singleLimb_ratios, singleLimb_labels)])):
    bottom -= height
    bc = ax2.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.20 * j)
    ax2.bar_label(bc, labels=[f"{height:.0%}"], label_type='center', fontsize = 21)

ax2.set_title('Focal movements', fontsize = 21)
ax2.legend(frameon = True, fancybox = True, shadow = True, fontsize = 16)
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

# use ConnectionPatch to draw lines between the two plots
theta1, theta2 = wedges[0].theta1, wedges[0].theta2
center, r = wedges[0].center, wedges[0].r
bar_height = sum(singleLimb_ratios)

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = r * np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(-width / 2, bar_height), coordsA=ax2.transData,
                      xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
con.set_linewidth(4)
ax2.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = r * np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax2.transData,
                        xyB=(x, y), coordsB=ax1.transData, linestyle = "-")
con.set_color([0, 0, 0])
ax2.add_artist(con)
con.set_linewidth(4)

# plt.suptitle("Wake", fontsize = 21)

plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/pie/movement_distribution_wake.png", dpi = 300, bbox_inches = 'tight')

In [184]:
# same during spt
singleLimb_df = bursts_df[bursts_df["Category"] == "Focal"].reset_index(drop=True)
perc_LW = singleLimb_df[singleLimb_df["Limbs"] == {"LW"}].shape[0] / singleLimb_df.shape[0]
perc_RW = singleLimb_df[singleLimb_df["Limbs"] == {"RW"}].shape[0] / singleLimb_df.shape[0]
perc_LL = singleLimb_df[singleLimb_df["Limbs"] == {"LL"}].shape[0] / singleLimb_df.shape[0]
perc_RL = singleLimb_df[singleLimb_df["Limbs"] == {"RL"}].shape[0] / singleLimb_df.shape[0]
perc_T = singleLimb_df[singleLimb_df["Limbs"] == {"T"}].shape[0] / singleLimb_df.shape[0]

# do what above but with the spt data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9), gridspec_kw={'width_ratios': [2, 1]})
fig.subplots_adjust(wspace=0)

# pie chart parameters
overall_ratios_spt = df["SPT"].values
labels_spt = df.index.to_list()
labels_spt[4] = "\n\n\nUpper\nBody"

explode = [0.1, 0, 0, 0, 0]
# rotate so that first wedge is split by the x-axis
angle = -45
wedges, *_ = ax1.pie(overall_ratios_spt, autopct='%1.1f%%', startangle=angle,
                     labels=labels_spt, explode=explode, textprops = {"fontsize": 21})

# bar chart parameters
singleLimb_ratios = [perc_LW, perc_RW, perc_RL, perc_LL, perc_T]
singleLimb_labels = ["LW", "RW", "RA", "LA", "T"]
bottom = 1
width = .2

# Adding from the top matches the legend.

for j, (height, label) in enumerate(reversed([*zip(singleLimb_ratios, singleLimb_labels)])):
    bottom -= height
    bc = ax2.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.20 * j)
    ax2.bar_label(bc, labels=[f"{height:.0%}"], label_type='center', fontsize = 21)

ax2.set_title('Focal movements', fontsize = 21)
ax2.legend(frameon = True, fancybox = True, shadow = True, fontsize = 16)
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

# use ConnectionPatch to draw lines between the two plots
theta1, theta2 = wedges[0].theta1, wedges[0].theta2
center, r = wedges[0].center, wedges[0].r
bar_height = sum(singleLimb_ratios)

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = r * np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(-width / 2, bar_height), coordsA=ax2.transData,
                      xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
con.set_linewidth(4)
ax2.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = r * np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax2.transData,
                        xyB=(x, y), coordsB=ax1.transData, linestyle = "-")
con.set_color([0, 0, 0])
ax2.add_artist(con)
con.set_linewidth(4)

# plt.suptitle("dSPT", fontsize = 21)
plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/pie/movement_distribution_spt.png", dpi = 300, bbox_inches = 'tight')


# 1st part vs 2nd part dSPT

In [158]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects}
tst_1st_half = {sub: 0 for sub in subjects}
tst_2nd_half = {sub: 0 for sub in subjects}

twt = {sub: 0 for sub in subjects}
twt_1st_half = {sub: 0 for sub in subjects}
twt_2nd_half = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df_1st_half = pd.DataFrame()
bursts_df_2nd_half = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)

    start_sleep, end_sleep = diary_SPT[sub]
    sleep_midpoint = start_sleep + (end_sleep - start_sleep) / 2

    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 3600
    tst_1st_half[sub] = (SIB[sub].loc[SIB[sub]["sib.onset.time"] < sleep_midpoint, "sib.duration"][:-1].sum() + (sleep_midpoint - SIB[sub].loc[SIB[sub]["sib.onset.time"] < sleep_midpoint, "sib.onset.time"].iloc[-1])).total_seconds() / 3600 
    tst_2nd_half[sub] = (SIB[sub].loc[SIB[sub]["sib.onset.time"] >= sleep_midpoint, "sib.duration"].sum() + (SIB[sub].loc[SIB[sub]["sib.onset.time"] >= sleep_midpoint, "sib.onset.time"].iloc[0] - sleep_midpoint)).total_seconds() / 3600

    twt[sub] = (SIB[sub]["awake.duration"].sum()).total_seconds() / 3600
    twt_1st_half[sub] = (SIB[sub].loc[SIB[sub]["sib.onset.time"] < sleep_midpoint, "awake.duration"].sum()).total_seconds() / 3600
    twt_2nd_half[sub] = (SIB[sub].loc[SIB[sub]["sib.onset.time"] >= sleep_midpoint, "awake.duration"].sum()).total_seconds() / 3600

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    df_merged_intervals_1st_half = df_merged_intervals.loc[df_merged_intervals["Start"] < sleep_midpoint].reset_index(drop=True)
    df_merged_intervals_2nd_half = df_merged_intervals.loc[df_merged_intervals["Start"] >= sleep_midpoint].reset_index(drop=True)

    bursts_df_1st_half = pd.concat([bursts_df_1st_half, df_merged_intervals_1st_half])
    bursts_df_2nd_half = pd.concat([bursts_df_2nd_half, df_merged_intervals_2nd_half])

tst_df = pd.DataFrame(tst, index = ["TST"]).T.sort_index()
tst_1st_half_df = pd.DataFrame(tst_1st_half, index = ["TST_1st_half"]).T.sort_index()
tst_2nd_half_df = pd.DataFrame(tst_2nd_half, index = ["TST_2nd_half"]).T.sort_index()

twt_df = pd.DataFrame(twt, index = ["TWT"]).T.sort_index()
twt_1st_half_df = pd.DataFrame(twt_1st_half, index = ["TWT_1st_half"]).T.sort_index()
twt_2nd_half_df = pd.DataFrame(twt_2nd_half, index = ["TWT_2nd_half"]).T.sort_index()

spt_df = pd.DataFrame([tst_df["TST"], twt_df["TWT"]], index = ["TST", "TWT"]).T
spt_dur_df = spt_df.sum(axis = 1)

spt_dur_half_df = spt_dur_df/2

In [159]:
bursts_df_sleep1 = bursts_df_1st_half[bursts_df_1st_half["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep1["Duration"] = bursts_df_sleep1["End"] - bursts_df_sleep1["Start"]
bursts_df_sleep1["Duration"] = bursts_df_sleep1["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep1["n_limbs"] = bursts_df_sleep1["Limbs"].apply(lambda x: len(x))
bursts_df_sleep1["Category"] = "placeholder"
bursts_df_sleep1.loc[bursts_df_sleep1["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_sleep1.loc[(bursts_df_sleep1["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep1["Limbs"] == {"LL", "T"}) | (bursts_df_sleep1["Limbs"] == {"RL", "T"}) | (bursts_df_sleep1["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_sleep1.loc[(bursts_df_sleep1["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep1["Limbs"] == {"LW", "T"}) | (bursts_df_sleep1["Limbs"] == {"RW", "T"}) | (bursts_df_sleep1["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_sleep1.loc[(bursts_df_sleep1["n_limbs"] >= 2) & (bursts_df_sleep1["n_limbs"] != 5) & (bursts_df_sleep1["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep1["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep1["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep1["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep1["Limbs"] != {"LL", "T"}) & (bursts_df_sleep1["Limbs"] != {"RL", "T"}) & (bursts_df_sleep1["Limbs"] != {"LW", "T"}) & (bursts_df_sleep1["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_sleep1.loc[bursts_df_sleep1["n_limbs"] == 1, "Category"] = "Focal"


bursts_df_sleep2 = bursts_df_2nd_half[bursts_df_2nd_half["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep2["Duration"] = bursts_df_sleep2["End"] - bursts_df_sleep2["Start"]
bursts_df_sleep2["Duration"] = bursts_df_sleep2["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep2["n_limbs"] = bursts_df_sleep2["Limbs"].apply(lambda x: len(x))
bursts_df_sleep2["Category"] = "placeholder"
bursts_df_sleep2.loc[bursts_df_sleep2["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_sleep2.loc[(bursts_df_sleep2["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep2["Limbs"] == {"LL", "T"}) | (bursts_df_sleep2["Limbs"] == {"RL", "T"}) | (bursts_df_sleep2["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_sleep2.loc[(bursts_df_sleep2["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep2["Limbs"] == {"LW", "T"}) | (bursts_df_sleep2["Limbs"] == {"RW", "T"}) | (bursts_df_sleep2["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_sleep2.loc[(bursts_df_sleep2["n_limbs"] >= 2) & (bursts_df_sleep2["n_limbs"] != 5) & (bursts_df_sleep2["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep2["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep2["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep2["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep2["Limbs"] != {"LL", "T"}) & (bursts_df_sleep2["Limbs"] != {"RL", "T"}) & (bursts_df_sleep2["Limbs"] != {"LW", "T"}) & (bursts_df_sleep2["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_sleep2.loc[bursts_df_sleep2["n_limbs"] == 1, "Category"] = "Focal"

bursts_df_sleep1.shape[0], bursts_df_sleep2.shape[0]

(705, 993)

In [160]:
bursts_df_1st_half["Duration"] = bursts_df_1st_half["End"] - bursts_df_1st_half["Start"]
bursts_df_1st_half["Duration"] = bursts_df_1st_half["Duration"].apply(lambda x: x.total_seconds())
bursts_df_1st_half["n_limbs"] = bursts_df_1st_half["Limbs"].apply(lambda x: len(x))
bursts_df_1st_half["Category"] = "placeholder"
bursts_df_1st_half.loc[bursts_df_1st_half["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_1st_half.loc[(bursts_df_1st_half["Limbs"] == {"LL", "RL"}) | (bursts_df_1st_half["Limbs"] == {"LL", "T"}) | (bursts_df_1st_half["Limbs"] == {"RL", "T"}) | (bursts_df_1st_half["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_1st_half.loc[(bursts_df_1st_half["Limbs"] == {"LW", "RW"}) | (bursts_df_1st_half["Limbs"] == {"LW", "T"}) | (bursts_df_1st_half["Limbs"] == {"RW", "T"}) | (bursts_df_1st_half["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_1st_half.loc[(bursts_df_1st_half["n_limbs"] >= 2) & (bursts_df_1st_half["n_limbs"] != 5) & (bursts_df_1st_half["Limbs"] != {"LL", "RL"}) & (bursts_df_1st_half["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_1st_half["Limbs"] != {"LW", "RW"}) & (bursts_df_1st_half["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_1st_half["Limbs"] != {"LL", "T"}) & (bursts_df_1st_half["Limbs"] != {"RL", "T"}) & (bursts_df_1st_half["Limbs"] != {"LW", "T"}) & (bursts_df_1st_half["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_1st_half.loc[bursts_df_1st_half["n_limbs"] == 1, "Category"] = "Focal"

bursts_df_2nd_half["Duration"] = bursts_df_2nd_half["End"] - bursts_df_2nd_half["Start"]
bursts_df_2nd_half["Duration"] = bursts_df_2nd_half["Duration"].apply(lambda x: x.total_seconds())
bursts_df_2nd_half["n_limbs"] = bursts_df_2nd_half["Limbs"].apply(lambda x: len(x))
bursts_df_2nd_half["Category"] = "placeholder"
bursts_df_2nd_half.loc[bursts_df_2nd_half["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_2nd_half.loc[(bursts_df_2nd_half["Limbs"] == {"LL", "RL"}) | (bursts_df_2nd_half["Limbs"] == {"LL", "T"}) | (bursts_df_2nd_half["Limbs"] == {"RL", "T"}) | (bursts_df_2nd_half["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_2nd_half.loc[(bursts_df_2nd_half["Limbs"] == {"LW", "RW"}) | (bursts_df_2nd_half["Limbs"] == {"LW", "T"}) | (bursts_df_2nd_half["Limbs"] == {"RW", "T"}) | (bursts_df_2nd_half["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_2nd_half.loc[(bursts_df_2nd_half["n_limbs"] >= 2) & (bursts_df_2nd_half["n_limbs"] != 5) & (bursts_df_2nd_half["Limbs"] != {"LL", "RL"}) & (bursts_df_2nd_half["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_2nd_half["Limbs"] != {"LW", "RW"}) & (bursts_df_2nd_half["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_2nd_half["Limbs"] != {"LL", "T"}) & (bursts_df_2nd_half["Limbs"] != {"RL", "T"}) & (bursts_df_2nd_half["Limbs"] != {"LW", "T"}) & (bursts_df_2nd_half["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_2nd_half.loc[bursts_df_2nd_half["n_limbs"] == 1, "Category"] = "Focal"

bursts_df_sleep1 = bursts_df_1st_half[bursts_df_1st_half["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep2 = bursts_df_2nd_half[bursts_df_2nd_half["SIB"] == 1].reset_index(drop=True)

bursts_df_wake1 = bursts_df_1st_half[bursts_df_1st_half["SIB"] == 0].reset_index(drop=True)
bursts_df_wake2 = bursts_df_2nd_half[bursts_df_2nd_half["SIB"] == 0].reset_index(drop=True)

Movement index

In [172]:
freq_1_sleep = bursts_df_sleep1.groupby(["Category", "sub_ID"])["Duration"].count() #/ (tst_1st_half_df["TST_1st_half"])
freq_2_sleep = bursts_df_sleep2.groupby(["Category", "sub_ID"])["Duration"].count() #/ (tst_2nd_half_df["TST_2nd_half"])
freq_1_sleep = freq_1_sleep.unstack().T.div(tst_1st_half_df["TST_1st_half"], axis = 0).T
freq_2_sleep = freq_2_sleep.unstack().T.div(tst_2nd_half_df["TST_2nd_half"], axis = 0).T

freq_1_wake = bursts_df_wake1.groupby(["Category", "sub_ID"])["Duration"].count() #/ (twt_1st_half_df["TWT_1st_half"])
freq_2_wake = bursts_df_wake2.groupby(["Category", "sub_ID"])["Duration"].count() #/ (twt_2nd_half_df["TWT_2nd_half"])
freq_1_wake = freq_1_wake.unstack().T.div(twt_1st_half_df["TWT_1st_half"], axis = 0).T
freq_2_wake = freq_2_wake.unstack().T.div(twt_2nd_half_df["TWT_2nd_half"], axis = 0).T

freq_1_spt = bursts_df_1st_half.groupby(["Category", "sub_ID"])["Duration"].count() #/ (spt_dur_half_df)
freq_2_spt = bursts_df_2nd_half.groupby(["Category", "sub_ID"])["Duration"].count() #/ (spt_dur_half_df)
freq_1_spt = freq_1_spt.unstack().T.div(spt_dur_half_df, axis = 0).T
freq_2_spt = freq_2_spt.unstack().T.div(spt_dur_half_df, axis = 0).T

In [166]:
freq_1_spt = bursts_df_1st_half.groupby("sub_ID")["Duration"].count() / (spt_dur_half_df)
print(freq_1_spt.mean(), freq_1_spt.std())
freq_2_spt = bursts_df_2nd_half.groupby("sub_ID")["Duration"].count() / (spt_dur_half_df)
freq_2_spt.mean(), freq_2_spt.std()

22.490568995630042 6.952919829753913


(32.996480888998924, 10.149218212596027)

Duration or AUC

In [163]:
freq_1 = bursts_df_sleep1.groupby(["Category", "sub_ID"])["AUC"].mean()
freq_2 = bursts_df_sleep2.groupby(["Category", "sub_ID"])["AUC"].mean()
freq_1 = freq_1.unstack()/1000
freq_2 = freq_2.unstack()/1000

In [244]:
freq

,part,sub_ID,Category,Frequency
0,1st half,098,Focal,5.657143
1,1st half,127,Focal,12.207888
2,1st half,158,Focal,7.384615
3,1st half,279,Focal,8.888889
4,1st half,547,Focal,5.794155
...,...,...,...,...
115,2nd half,906,Mixed,3.913043
116,2nd half,914,Mixed,2.809507
117,2nd half,958,Mixed,3.424181
118,2nd half,965,Mixed,4.807122


In [171]:
freq_1_spt

sub_ID
098    20.314286
127    27.600442
158    18.104218
279    14.722222
547    15.247776
633    18.015531
815    37.374046
906    27.391304
914    18.644910
958    26.339857
965    29.109792
971    17.022443
dtype: float64

In [170]:
freq_1_sleep

sub_ID,098,127,158,279,547,633,815,906,914,958,965,971
Category,,,,,,,,,,,,
Focal,4.845606,12.867355,7.886926,8.728979,5.848653,0.948200,8.458150,9.846664,6.612785,7.503257,10.119884,5.329573
Full Body,1.995249,4.003177,4.070671,2.252640,2.154767,7.901668,11.277533,3.580605,2.909625,5.314807,2.381149,2.537892
Lower Body,3.990499,0.285941,1.017668,1.407900,3.693886,2.212467,1.057269,0.895151,1.587068,0.937907,3.869368,2.791681
Mixed,2.565321,4.575060,0.254417,0.844740,1.231295,3.160667,2.466960,2.387070,1.322557,5.002171,4.464655,2.030314
Upper Body,1.710214,NaN,1.017668,0.281580,0.923472,0.632133,1.762115,2.983838,0.529023,2.501086,1.190575,0.507578


In [173]:
# merge and boxplot each movement 1st vs 2nd half
freq_sleep = pd.concat([freq_1_sleep, freq_2_sleep], axis = 1).T
freq_sleep["part"] = "1st half"
freq_sleep.reset_index(drop=True, inplace = True)
freq_sleep.loc[12:, "part"] = "2nd half"
freq_sleep["sub_ID"] = sorted(subjects) + sorted(subjects)

freq_wake = pd.concat([freq_1_wake, freq_2_wake], axis = 1).T
freq_wake["part"] = "1st half"
freq_wake.reset_index(drop=True, inplace = True)
freq_wake.loc[12:, "part"] = "2nd half"
freq_wake["sub_ID"] = sorted(subjects) + sorted(subjects)

freq_spt = pd.concat([freq_1_spt, freq_2_spt], axis = 1).T
freq_spt["part"] = "1st half"
freq_spt.reset_index(drop=True, inplace = True)
freq_spt.loc[12:, "part"] = "2nd half"
freq_spt["sub_ID"] = sorted(subjects) + sorted(subjects)

# freq_sleep = freq_sleep.unstack().T
# freq_sleep["part"] = freq_sleep.index.get_level_values(0)
# freq_sleep["sub_ID"] = freq_sleep.index.get_level_values(1)
# freq_sleep.reset_index(drop = True, inplace = True)

freq_sleep = freq_sleep.melt(id_vars = ["part", "sub_ID"], value_vars = [ "Focal", "Lower Body", "Upper Body", "Mixed", "Full Body"], var_name = "Category", value_name = "Freq_sleepuency")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15, 9))
sns.boxplot(x = "Category", y = "Freq_sleepuency", hue = "part", data = freq_sleep, showfliers = False, ax = ax2)
sns.stripplot(x = "Category", y = "Freq_sleepuency", hue = "part", data = freq_sleep, dodge = True, jitter = True, palette = "dark:black", alpha = 0.52, s = 7, ax = ax2, legend = False)
ax2.set_xlabel("Category", fontsize = 19)
ax2.set_ylabel("Movement index (per hour of aS)", fontsize = 19)
ax2.legend(fontsize = 19, frameon = True, fancybox = True, shadow = True, loc = "upper center")
ax2.set_xlabel("")
ax2.xaxis.set_tick_params(rotation = 41, labelsize = 16)
ax2.yaxis.set_tick_params(labelsize = 16)
ax2.set_ylim(0,30)
ax2.set_title("aS", fontsize = 21)

freq_wake = freq_wake.melt(id_vars = ["part", "sub_ID"], value_vars = [ "Focal", "Lower Body", "Upper Body", "Mixed", "Full Body"], var_name = "Category", value_name = "Freq_wakeuency")

sns.boxplot(x = "Category", y = "Freq_wakeuency", hue = "part", data = freq_wake, showfliers = False, ax = ax3)
sns.stripplot(x = "Category", y = "Freq_wakeuency", hue = "part", data = freq_wake, dodge = True, jitter = True, palette = "dark:black", alpha = 0.52, s = 7, ax = ax3, legend = False)
ax3.set_xlabel("Category", fontsize = 19)
# ax3.legend(fontsize = 19, frameon = True, fancybox = True, shadow = True, loc = "upper center")
ax3.set_ylabel("Movement index (per hour of aW)", fontsize = 19)
ax3.set_xlabel("")
ax3.xaxis.set_tick_params(rotation = 41, labelsize = 16)
ax3.yaxis.set_tick_params(labelsize = 16)
ax3.set_ylim(0,100)
ax3.set_title("aW", fontsize = 21)
# dont show legend
ax3.get_legend().remove()

freq_spt = freq_spt.melt(id_vars = ["part", "sub_ID"], value_vars = [ "Focal", "Lower Body", "Upper Body", "Mixed", "Full Body"], var_name = "Category", value_name = "Freq_sptuency")

sns.boxplot(x = "Category", y = "Freq_sptuency", hue = "part", data = freq_spt, showfliers = False, ax = ax1)
sns.stripplot(x = "Category", y = "Freq_sptuency", hue = "part", data = freq_spt, dodge = True, jitter = True, palette = "dark:black", alpha = 0.52, s = 7, ax = ax1, legend = False)
ax1.set_xlabel("Category", fontsize = 19)
# ax1.legend(fontsize = 19, frameon = True, fancybox = True, shadow = True, loc = "upper center")
ax1.set_ylabel("Movement index (per hour of dSPT)", fontsize = 19)
ax1.set_xlabel("")
ax1.xaxis.set_tick_params(rotation = 41, labelsize = 16)
ax1.yaxis.set_tick_params(labelsize = 16)
ax1.set_ylim(0,30)
ax1.set_title("dSPT", fontsize = 21)
# dont show legend
ax1.get_legend().remove()

plt.tight_layout()

# plt.ylim(0, freq_sleep["Freq_sleepuency"].max() + 5)

In [270]:
plt.savefig("/Users/marcellosicbaldi/Desktop/paper-Sleep/figures_v2/1st_vs_2nd_halves.png", dpi = 300, bbox_inches = 'tight')

In [175]:
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests
from scipy.stats import shapiro
p_values_shapiro_sleep = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    # Before doing the test, check if the data is normally distributed using the Shapiro-Wilk test
    p_values_shapiro_sleep[cat] = shapiro(freq_sleep.loc[(freq_sleep["Category"] == cat) & (freq_sleep["part"] == "1st half"), "Freq_sleepuency"])[1], shapiro(freq_sleep.loc[(freq_sleep["Category"] == cat) & (freq_sleep["part"] == "2nd half"), "Freq_sleepuency"])[1]

p_values_shapiro_sleep

{'Full Body': (0.0042622885666787624, 0.095077283680439),
 'Lower Body': (0.09461544454097748, 0.46995437145233154),
 'Upper Body': (1.0, 1.0),
 'Mixed': (0.551009476184845, 0.9613924622535706),
 'Focal': (0.9710031151771545, 0.5009877681732178)}

In [181]:
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests
from scipy.stats import shapiro
p_values_shapiro_wake = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    # Before doing the test, check if the data is normally distributed using the Shapiro-Wilk test
    p_values_shapiro_wake[cat] = shapiro(freq_wake.loc[(freq_wake["Category"] == cat) & (freq_wake["part"] == "1st half"), "Freq_wakeuency"])[1], shapiro(freq_wake.loc[(freq_wake["Category"] == cat) & (freq_wake["part"] == "2nd half"), "Freq_wakeuency"])[1]

p_values_shapiro_wake

{'Full Body': (2.5284874936915003e-05, 0.34253954887390137),
 'Lower Body': (1.0, 1.0),
 'Upper Body': (1.0, 1.0),
 'Mixed': (1.0, 1.0),
 'Focal': (1.0, 0.20635491609573364)}

In [182]:
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests
from scipy.stats import shapiro
p_values_shapiro_spt = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    # Before doing the test, check if the data is normally distributed using the Shapiro-Wilk test
    p_values_shapiro_spt[cat] = shapiro(freq_spt.loc[(freq_spt["Category"] == cat) & (freq_spt["part"] == "1st half"), "Freq_sptuency"])[1], shapiro(freq_spt.loc[(freq_spt["Category"] == cat) & (freq_spt["part"] == "2nd half"), "Freq_sptuency"])[1]

p_values_shapiro_spt

{'Full Body': (0.007103098556399345, 0.027962788939476013),
 'Lower Body': (0.07124915719032288, 0.30847033858299255),
 'Upper Body': (1.0, 1.0),
 'Mixed': (0.2095041126012802, 0.9033188819885254),
 'Focal': (0.777447521686554, 0.4506551921367645)}

In [176]:
p_values_shapiro_sleep = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    p_1st_half = shapiro(freq_sleep.loc[(freq_sleep["Category"] == cat) & (freq_sleep["part"] == "1st half"), "Freq_sleepuency"])[1]
    p_2nd_half = shapiro(freq_sleep.loc[(freq_sleep["Category"] == cat) & (freq_sleep["part"] == "2nd half"), "Freq_sleepuency"])[1]
    
    # Check normality assumption for both halves
    if p_1st_half > 0.05 and p_2nd_half > 0.05:
        print(f"{cat}: Data is normally distributed in both halves. Use a parametric test.")
    else:
        print(f"{cat}: Data is NOT normally distributed in at least one half. Use a non-parametric test.")

Full Body: Data is NOT normally distributed in at least one half. Use a non-parametric test.
Lower Body: Data is normally distributed in both halves. Use a parametric test.
Upper Body: Data is normally distributed in both halves. Use a parametric test.
Mixed: Data is normally distributed in both halves. Use a parametric test.
Focal: Data is normally distributed in both halves. Use a parametric test.


In [177]:
p_values_shapiro_wake = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    p_1st_half = shapiro(freq_wake.loc[(freq_wake["Category"] == cat) & (freq_wake["part"] == "1st half"), "Freq_wakeuency"])[1]
    p_2nd_half = shapiro(freq_wake.loc[(freq_wake["Category"] == cat) & (freq_wake["part"] == "2nd half"), "Freq_wakeuency"])[1]

    # Check normality assumption for both halves
    if p_1st_half > 0.05 and p_2nd_half > 0.05:
        print(f"{cat}: Data is normally distributed in both halves. Use a parametric test.")
    else:
        print(f"{cat}: Data is NOT normally distributed in at least one half. Use a non-parametric test.")

Full Body: Data is NOT normally distributed in at least one half. Use a non-parametric test.
Lower Body: Data is normally distributed in both halves. Use a parametric test.
Upper Body: Data is normally distributed in both halves. Use a parametric test.
Mixed: Data is normally distributed in both halves. Use a parametric test.
Focal: Data is normally distributed in both halves. Use a parametric test.


In [178]:
p_values_shapiro_spt = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    p_1st_half = shapiro(freq_spt.loc[(freq_spt["Category"] == cat) & (freq_spt["part"] == "1st half"), "Freq_sptuency"])[1]
    p_2nd_half = shapiro(freq_spt.loc[(freq_spt["Category"] == cat) & (freq_spt["part"] == "2nd half"), "Freq_sptuency"])[1]

    # Check normality assumption for both halves
    if p_1st_half > 0.05 and p_2nd_half > 0.05:
        print(f"{cat}: Data is normally distributed in both halves. Use a parametric test.")
    else:
        print(f"{cat}: Data is NOT normally distributed in at least one half. Use a non-parametric test.")

Full Body: Data is NOT normally distributed in at least one half. Use a non-parametric test.
Lower Body: Data is normally distributed in both halves. Use a parametric test.
Upper Body: Data is normally distributed in both halves. Use a parametric test.
Mixed: Data is normally distributed in both halves. Use a parametric test.
Focal: Data is normally distributed in both halves. Use a parametric test.


In [186]:
# Now, do the test! All t-test except full-body, which is a Wilcoxon test
p_values_sleep = {}
p_values_wake = {}
p_values_spt = {}

for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    p_1st_half = freq_sleep.loc[(freq_sleep["Category"] == cat) & (freq_sleep["part"] == "1st half"), "Freq_sleepuency"]
    p_2nd_half = freq_sleep.loc[(freq_sleep["Category"] == cat) & (freq_sleep["part"] == "2nd half"), "Freq_sleepuency"]
    if cat == "Full Body":
        p_values_sleep[cat] = wilcoxon(p_1st_half, p_2nd_half, nan_policy="omit")[1]
    else:
        p_values_sleep[cat] = ttest_rel(p_1st_half, p_2nd_half, nan_policy="omit")[1]

    p_1st_half = freq_wake.loc[(freq_wake["Category"] == cat) & (freq_wake["part"] == "1st half"), "Freq_wakeuency"]
    p_2nd_half = freq_wake.loc[(freq_wake["Category"] == cat) & (freq_wake["part"] == "2nd half"), "Freq_wakeuency"]
    if cat == "Full Body":
        p_values_wake[cat] = wilcoxon(p_1st_half, p_2nd_half, nan_policy="omit")[1]
    else:
        p_values_wake[cat] = ttest_rel(p_1st_half, p_2nd_half, nan_policy="omit")[1]

    p_1st_half = freq_spt.loc[(freq_spt["Category"] == cat) & (freq_spt["part"] == "1st half"), "Freq_sptuency"]
    p_2nd_half = freq_spt.loc[(freq_spt["Category"] == cat) & (freq_spt["part"] == "2nd half"), "Freq_sptuency"]
    if cat == "Full Body":
        p_values_spt[cat] = wilcoxon(p_1st_half, p_2nd_half, nan_policy="omit")[1]
    else:
        p_values_spt[cat] = ttest_rel(p_1st_half, p_2nd_half, nan_policy="omit")[1]

In [187]:
p_values_sleep

{'Full Body': 0.02099609375,
 'Lower Body': 0.005253378722886183,
 'Upper Body': 0.031167061186783352,
 'Mixed': 0.0659686181737909,
 'Focal': 0.08365936093403954}

In [188]:
p_values_wake

{'Full Body': 0.5693359375,
 'Lower Body': 0.4614259840435855,
 'Upper Body': 0.7571062422354079,
 'Mixed': 0.5540011533165363,
 'Focal': 0.6435472119543435}

In [189]:
p_values_spt

{'Full Body': 0.00244140625,
 'Lower Body': 0.005206500522422952,
 'Upper Body': 0.0664764114480816,
 'Mixed': 0.04788353570270041,
 'Focal': 0.07622829135880047}